## ------
# This model corresponds to the magma ocean model (sans basal magma ocean) within the manuscript Moore, Cowan, & Boukare (2023, MNRAS). For a thorough model description, including all equations, assumptions, and parameters, please refer to the paper.
## ------

# ** NOTE: CHANGE save_path BEFORE RUNNING! **

# TESTED PARAMETER SPACE:
## Host star = {M8, M5, M1} (Baraffe+ 15)
## Water inventory = {2, 4, 6, 8} Earth Oceans
## tau_MO = tau_RG IN ALL CASES
## Orbital distance = {Hot Inner, Mid, Cold Outer} HZ edge (Kopparapu+ 13)
# ---------------------------------

# ASSUMPTIONS:
## MAGMA OCEAN PHASE IS SAME AS RUNAWAY GREENHOUSE PHASE --- USE THE LATTER AS THE TIMESCALE FOR THIS TIME (calculated using irradiation at orbital distance of planet)
## MINIMUM VALUE OF WATER IN MANTLE SAME AS SURFACE DESICCATION LIMIT, TO AVOID ETA --> INFINITY!!!! 

## Luger & Barnes (2015) assume 10 Myr formation time; Earth's magma ocean phase ~1.5 Myr; this is extended around M dwarfs (Lebrun+13)! Barth+(2020) assume a disk lifetime 5 Myr at beginning of magma ocean phase, and show solidification timescales for different water inventories --- WE ADOPT THIS 5 Myr OFFSET TO THE STELLAR TRACK, TO ALLOW PLANETARY FORMATION.
## - CONSTANT thermospheric temperature = 400 K (diffusion-limited escape)
## - Pure H2O atmosphere
## Runaway greenhouse limit: we use the 'water condensation insolation threshold' of 325 W/m^2 (Turbet+2021)

######################################################################################################################

# Initialize notebook; import required functions/packages, and read in input parameters/variables from .txt files.

In [ ]:
%pylab inline
import math
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from pylab import *
from scipy.integrate import ode
#from mpl_toolkits.axes_grid.inset_locator import inset_axes
import time
import os.path

In [ ]:
# Need to divide constants into three categories --

# 1) Unchanging constants
# [M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry]

params1 = np.loadtxt('const_params.txt')
M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1

# 2) Varied parameters
# [x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
#  T_serp, K_cnst, gamma, r_fug, d_b]

params2 = np.loadtxt('vary_params.txt')
x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
     T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
# Test different betas; see if the code breaks
#beta = 0.12
#params2[11] = beta

# 3) Changing parameters
# [omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt]

params3 = np.loadtxt('change_params.txt')
omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
dt_nom = 2.0e4*year
params3[8] = dt_nom

# NEW constants, for 1-D atmosphere calculations
sigma_sb = 5.67e-8 #[W/m^2/K^4]; Stefan-Boltzmann constant
m_H = 1.66e-27 #[kg]; mass of H atom
k_B = 1.38e-23 #[m^2 kg s^-2 K^-1]; Boltzmann constant
m_H2O = 18.02*m_H #molecular mass of water
m_air = 28.7*m_H #molecular mass of air on Earth
rho_water = 997. #[kg/m^3]; density of water
cp_water_vapour = 1.996e3 #[J/(kg K)]; specific heat capacity of water vapour (steam)

In [ ]:
# Parameters of orbiting planet (can be changed later).
alb = 0.3 #albedo; value roughly similar to Earth
#alb = 0.75 #albedo; upper limit to roughly test "steam atmosphere"

In [ ]:
# File path of where to save files and find input files.
save_path = '/Users/admin/Desktop/Model2_Paper2/'

# Model Functions/Calculated Constants

In [ ]:
## WATER FUGACITY
# Need to calculate non-dimensional water fugacity at each step.
# Formula original from Li et al. (2008), based on experimental data.
def lnf_w(x, params1, params2, params3): #NOTE: Takes non-dimensionalized mantle water mass fraction and converts it.
    c0 = -7.9859
    c1 = 4.3559
    c2 = -0.5742
    c3 = 0.0337
    B = 2.0e6
    mu_oliv = 153.31
    mu_wat = 18.01528
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    ln_term = np.log((B*x*(mu_oliv/mu_wat))/(1.-(x*(mu_oliv/mu_wat))))
    
    return c0 + c1*ln_term + c2*ln_term**2. + c3*ln_term**3.

def f_w(x, params1, params2, params3):
    
    return np.exp(lnf_w(x, params1, params2, params3))

In [ ]:
## MANTLE VISCOSITY

# Need a function to calculate the viscosity.
def eta(temp, x, params1, params2, params3): #x and T are non-dimensionalized later in code -- need to add dimensions back for these calculations
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    eta_scale = (np.exp(lnf_w(5.8e-4,params1,params2,params3)))**(-r_fug)
    eta_0 = 1.0e21/eta_scale
        
    # Need a minimum fugacity, to avoid f_w --> 0 and eta --> infinity.
    # Let's take the 'desiccation limit' for the surface as the minimum value to be 'trapped' in the mantle.
    if x <= 1.29e16/(f_M*M_E):
        return eta_0*(f_w(1.29e16/(f_M*M_E), params1, params2, params3)**(-r_fug)) * np.exp((E_a/R_g)*((1./temp) - (1./T_ref)))
    else:
        return eta_0*(f_w(x, params1, params2, params3)**(-r_fug)) * np.exp((E_a/R_g)*((1./temp) - (1./T_ref)))

In [ ]:
# Planetary radius [m]
def Rp(M):
    M_E = 5.972e24
    R_E = 6.371e6
    return R_E*(M/M_E)**0.27

# Core radius [m]
def Rc(M):
    M_E = 5.972e24
    R_E = 6.371e6
    return 0.547*R_E*(M/M_E)**0.25

# Extent of mantle [m]
def h(M):
    return Rp(M) - Rc(M)

# Surface gravity [m/s^2]
def g(M):
    G = 6.67e-11
    return G*M/(Rp(M)**2.)

# Planetary surface area [m^2]
def A(M):
    return 4.*np.pi*(Rp(M)**2.)

# Mantle volume [m^3]
def V(M):
    return (4.*np.pi/3.)*((Rp(M)**3.) - (Rc(M)**3.))

def F_0(M, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    eta_scale = (np.exp(lnf_w(1.,params1,params2,params3)))**(-r_fug)
    eta_0 = 1.e21/eta_scale

    f_w = np.exp(lnf_w(5.8e-4,params1,params2,params3))
    return (k*(T_ref**(1.+beta))*A(M)/(h(M)*V(M)))*(alpha*rho_m*g(M)*(h(M)**3.)*f_w/(Ra_c*kappa*eta_0))**beta

# Mid-ocean ridge length [m]
def L_MOR(M):
    return 1.5*2.*np.pi*Rp(M)

# Mid-ocean ridge spreading rate [m/s]
def S(t, temp, x, T_surf, M, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return 10.76*(kappa**(1.-2.*beta))*(h(M)**(6.*beta-1.))*((alpha*rho_m*g(M)*\
            (temp-T_surf)/(eta(temp,x,params1,params2,params3)*Ra_c))**(2.*beta))

def tau(M, params1, params2, params3): #useful for converting between different non-dimensional timescales
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return (L_MOR(M)*(0.1/year)*x_h*rho_c*chi*d_hE)/(M*omega_0*f_btwid) #spreading rate independent of temp in this model

# Rayleigh number, related to convection
def Ra(t, temp, x, T_surf, M, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return (alpha*rho_m*g(M)*(temp - T_surf)*(h(M)**3))/(eta(temp, x, params1, params2, params3)*kappa)

# Nusselt number, for looking at heat flux
def Nu(temp, x, T_surf, M, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return (Ra(t, temp, x, T_surf, M, params1, params2, params3)/Ra_c)**beta

# Run MO simulation here.

In [ ]:
# Stellar evolution and surface temperature calculations

# Stellar evolution track, from Baraffe+ 2015
# Read in stellar evolution track data
hostfile = 'BHAC15_0.09Msun.txt' #M8; CHANGE FILE FOR DIFFERENT STELLAR HOSTS

def hoststar(file):
    
    data = np.loadtxt(file, skiprows=3)
    log_age = data[:,1] #log(yr)
    T_eff_star = data[:,2] #[K]
    Lbol_Ls = data[:,3] #log luminosity
    R_Rs = data[:,5]

    Ls = 3.839e33 # [erg/s]; solar bolometric luminosity
    Lbol = np.zeros(len(Lbol_Ls))
    Lbol_Ls_star = np.zeros(len(Lbol_Ls))
    for idx in range(0,len(Lbol_Ls)):    
        Lbol[idx] = (10.**Lbol_Ls[idx])*Ls/1.0e7 #[W]
        Lbol_Ls_star[idx] = 10.**Lbol_Ls[idx]
    
    Rs = 6.96e10 #[cm]; solar radius
    Rstar = (R_Rs*Rs)/100.0 #[m]

    star_age = np.zeros(len(log_age)) #[s], for later calculations

    # Convert stellar age to time, in seconds.
    for idx in range(0,len(log_age)):
        star_age[idx] = ((10.0**log_age[idx])*year)
    
    #print('Stellar age: ', star_age[0]/year, 'years')
    #print(star_age[0])
    
    return star_age, Lbol, Lbol_Ls_star, T_eff_star
    
# Calculate the top-of-atmosphere bolometric flux, from BHAC15 model.
def S_0(t, M, a_orb, params1, params2, params3):
        
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    t_int = t + (5.0e6*year) #add offset of 5 Myr, to account for formation
    
    return np.interp(t_int, star_age, Lbol)/(4.*np.pi*(a_orb**2.))
    
# Calculate evolution of XUV luminosity (Luger & Barnes 2015; Ribas+ 2005)
def L_XUV(t, M, params1, params2, params3):
        
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    t_int = t + (5.0e6*year) #add offset of 5 Myr, to account for formation
  
    f_sat = 1.0e-3 #saturation fraction
    t_sat = 1.0e9*year #[s], saturation timescale
    beta_XUV = -1.23
    
    if t <= t_sat:
        return f_sat*np.interp(t_int, star_age, Lbol)
    else: #t > t_sat
        return f_sat*((t_int/t_sat)**beta_XUV)*np.interp(t_int, star_age, Lbol)

# Calculate top-of-atmosphere XUV flux. 
def F_XUV(t, M, a_orb, params1, params2, params3):
        
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
        
    return L_XUV(t, M, params1, params2, params3)/(4.*np.pi*(a_orb**2.))

# Calculate the effective temperature of the planet.
def f_T_eff(t, M, a_orb, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return ((S_0(t, M, a_orb, params1, params2, params3)*(1.0-alb))/(4.0*sigma_sb))**(0.25)

# Calculate the skin temperature (i.e., the isothermal stratosphere temperature). [K]
def T_strat(t, M, a_orb, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return ((1./2.)**0.25)*((S_0(t, M, a_orb, params1, params2, params3)*(1.0-alb))/(4.0*sigma_sb))**(0.25)

In [ ]:
# Based on Fig. 3 of Barth+2020, Fig. 4 of Schaefer+2016:
# Magma ocean depth as a function of time
def d_MO(t, tau_MO, M): 
    
    d_MO_0 = h(M)/1000. # ENTIRE MANTLE BEGINS MOLTEN #[km]
    beta_MO = 1./(np.exp(1.)-1)
    
    return (beta_MO*d_MO_0)*(np.exp((-t/tau_MO) + 1.) - 1)

# Temperature of MO as a function of MO depth, using liquidus temperature (Eqn. 1 of Elkins-Tanton 2008)
def T_MO(t, tau_MO, M, a_orb, params1, params2, params3):
    
    r = (Rp(M) - d_MO(t, tau_MO, M))/1000. #[km]
    
    return (-1.16e-7*(r**3.)) + (0.0014*(r**2.)) - (6.382*r) + 1.444e4

# Solidus temperature (Eqn. 1 of Elkins-Tanton 2008)
def T_sol(r):
    
    return (-1.16e-7*(r**3.)) + (0.0014*(r**2.)) - (6.382*r) + 1.444e4

## Set the initial amount of water in the MO, and calculate the initial concentration.

In [ ]:
# Initial/constant parameters
D = 0.001 #0.001 OR 0.01 OR 0.1 #solid-liquid H2O distribution coefficient -- REQUIRED FOR THIS NOTEBOOK!!!
#D = 0.2 # higher value "mimics" behaviour of BMO; SEE BMO NOTEBOOK!!!
R_p = Rp(M_E)/1000. #[km] #6400 km
R_c = Rc(M_E)/1000. #[km] #3000 km
rho = 3.0e3*1.0e9 #[kg/km^3] 
rho_MO = rho #density of MO [kg/km^3] 

# Saturation concentration & initial concentration of water in MO
C_sat = 0.001 #NOMINAL; OR 0.1, OR 0.01 (degassing earlier with lower C_sat) #[wt fraction]; H2O saturation limit of MO

# Initial mass of water in MO, and initial concentration in the MO.
M_init_array = np.array([2., 4., 6., 8.])*1.4e21 #[kg]
C_0_array = (3.*M_init_array)/(4.*np.pi*rho*(R_p**3. - R_c**3.))

# Required functions -- RECALL (r = R_p - d_MO(t))
# Liquid concentration (in melt)
def C_l(C_0, r):
    
    return C_0*((R_p**3.-R_c**3.)/(R_p**3.-r**3.))**(1.-D)

# Solid concentration (in solid mantle)
def C_s(C_0, r):
    
    return D*C_l(C_0, r)

# Saturation radius
def R_sat(C_0):
    
    return (R_p**3. - ((R_p**3. - R_c**3.)/((C_sat/C_0)**(1./(1.-D)))))**(1./3.)

# Mass of water in unsaturated magma ocean
def M_MO_unsat(C_0, r):
    
    return C_l(C_0, r)*(4.*np.pi/3.)*rho_MO*(R_p**3. - r**3.)

# Mass of water in unsaturated solid mantle
def M_SM_unsat(C_0, r):
    
    return C_0*(4.*np.pi/3.)*(rho_MO)*((R_p**3. - R_c**3.)**(1.-D))*\
            (((R_p**3. - R_c**3.)**D) - ((R_p**3. - r**3.)**D))

# Mass of water in saturated magma ocean 
def M_MO_sat(r):
    
    return C_sat*(4.*np.pi/3.)*rho_MO*(R_p**3. - r**3.)

# Mass of water in saturated solid mantle
def M_SM_sat(C_0, r):
    
    return M_SM_unsat(C_0, R_sat(C_0)) + D*C_sat*(4.*np.pi/3.)*(rho_MO)*(r**3.-R_sat(C_0)**3.)

In [ ]:
def loss_rate_EL_MO(t, F_XUV, M): # energy-limited loss rate [kg/s]
    
    # R_XUV = XUV deposition radius; Luger & Barnes (2015) set R_XUV = R_p for simplicity
    # M = planetary mass
    
    G = 6.67e-11
    eps_XUV = 0.1 #nominal; Luger & Barnes (2015) test 0.15-0.30
    
    return eps_XUV*np.pi*F_XUV*(Rp(M)**3.)/(G*M)

def loss_rate_DL_MO(M): # diffusion-limited loss rate [kg/s]
    
    g_p = g(M)*100. #[cm s^-2], to match the units of b
    k_B = 1.38e-23 #[m^2 kg s^-2 K^-1]; Boltzmann constant
    
    # Mole fractions of H & O (assuming all H2O is dissociated)
    X_H = 2./3.
    X_O = 1./3.
    
    m_H = 1.66e-27 #[kg]; mass of H atom
    m_O = 16.*m_H
        
    # Just take the thermospheric temperature as 400 K at all times, for now.
    b = 4.8e17*(400.**0.75) #[cm^-1 s^-1]
        
    return m_H*(np.pi*(Rp(M)**2.)*b*g_p*(m_O-m_H))/(k_B*400.*(1.+(X_O/X_H))) 
    
def f_loss_MO(t, M, a_orb, params1, params2, params3): 
    
    ### Note that the "check" for not losing more water than on the surface is in the MO loop itself
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
   # Planet always in a runaway greenhouse during MO, so loss is always EL.
    
    return loss_rate_EL_MO(t, F_XUV(t, M, a_orb, params1, params2, params3), M) #[kg/s]

In [ ]:
## Orbital distances.
# Read in Kopparapu+(2013) HZ data, for different stellar hosts.

data = np.loadtxt('HZs.dat', skiprows=2)
T_eff_kopp = data[:,0] #[K]
# All below values give S_eff -- see Eqn.(2) of Kopparapu+(2013)
RV_kopp = data[:,1] #Recent Venus
RG_kopp = data[:,2] #Runaway Greenhouse
MG_kopp = data[:,3] #Maximum Greenhouse
EM_kopp = data[:,4] #Early Mars

def f_a_orb(t, M, T_eff_star, Lbol_Ls_star, params1, params2, params3):
    
    S_eff_RV = np.interp(T_eff_star, T_eff_kopp, RV_kopp)
    S_eff_RG = np.interp(T_eff_star, T_eff_kopp, RG_kopp)
    S_eff_MG = np.interp(T_eff_star, T_eff_kopp, MG_kopp)
    S_eff_EM = np.interp(T_eff_star, T_eff_kopp, EM_kopp)
    
    a_RV = (Lbol_Ls_star/S_eff_RV)**0.5 #[AU]
    a_RG = (Lbol_Ls_star/S_eff_RG)**0.5 #[AU]
    a_MG = (Lbol_Ls_star/S_eff_MG)**0.5 #[AU]
    a_EM = (Lbol_Ls_star/S_eff_EM)**0.5 #[AU]
    
    return a_RV, a_RG, a_MG, a_EM

In [ ]:
# Stellar parameters required for MO sims
star_age, Lbol, Lbol_Ls_star, T_eff_star = hoststar(hostfile)

# Roughly find limits at t = 4.5 Gyr (~age of Earth)
T_eff_star_45 = np.interp(4.5e9*year, star_age, T_eff_star)
Lbol_Ls_star_45 = np.interp(4.5e9*year, star_age, Lbol_Ls_star)
        
a_RV, a_RG, a_MG, a_EM = f_a_orb(4.5e9*year, M_E, T_eff_star_45, Lbol_Ls_star_45, params1, params2, params3)
    
# For now, take runaway greenhouse and maximum greenhouse limits, and then the middle between them.
a_orb_array = np.array([a_RG, (a_RG+a_MG)/2., a_MG])
a_orb_labels = ['Runaway_GH', 'Middle', 'Maximum_GH']

In [ ]:
# Calculate the runaway greenhouse duration, using the above orbital distances, 
# to be used as the magma ocean solidification timescale for each.

dt_MO = 2.0e3*year #2000 yr per timestep
M = M_E

RG_t_array = np.arange(0., 1.0e9*year, dt_MO) #[s]

tau_RG_array = np.zeros(len(a_orb_array)) #[s]
exit_RG_array = np.zeros(len(a_orb_array)) #index

for jdx in range(0, len(a_orb_array)):
    
    RG_TOA_flux_array = S_0(RG_t_array, M, a_orb_array[jdx]*1.496e11, params1, params2, params3)

    for idx in range(0, len(RG_t_array)):
    
        if RG_TOA_flux_array[idx]/4. < 325./(1.-alb):
        
            tau_RG_array[jdx] = RG_t_array[idx]
            exit_RG_array[jdx] = idx
            break
        
print(tau_RG_array/year/1.0e9, 'Gyr')

In [ ]:
# Set the duration of the MO to be the same as the duration of RG.
tau_MO_array = tau_RG_array
# tau_RG is tied to a given orbital distance!!!!

#M_init_array is the TOTAL initial water inventory, which starts in the MO.
M_init_file = np.array([2., 4., 6., 8.]) #[Earth Oceans]

In [ ]:
# Loop to track water in magma ocean (MO), solid mantle (SM) and atmosphere (atmo), over time.
# Within the loop, determine whether or not MO is saturated & degas an atmosphere if it is,
# from which water may be lost to space.
dt_MO = 2.0e3*year #2000 yr per timestep
M = M_E #planetary mass = mass of the Earth [kg]

# Arrays to save final water inventories at end of MO simulation.
W_m_init_array = np.zeros((len(M_init_array), len(a_orb_array)))
W_s_init_array = np.zeros((len(M_init_array), len(a_orb_array)))

# Loop over water inventories.
for kdx in range(0, len(M_init_array)): 
        
    # Loop over orbital distances + RG durations, tied together.
    for jdx in range(0,len(a_orb_array)):
            
        t_MO_array = np.arange(0., (tau_MO_array[jdx]+dt_MO), dt_MO) # 50 Myr; equivalent to MO solidification timescale
        r_array = R_p - d_MO(t_MO_array, tau_MO_array[jdx], M)
        
        save_file = 'model2_MO_only_loss_M_init-' + str(M_init_file[kdx]) + 'TO_' + str(a_orb_labels[jdx]) + '.txt'
            
        filename = os.path.join(save_path, save_file)
        f = open(filename, 'w')

        # 4 TOTAL reservoirs, for mass balance throughout
        M_MO_array = np.zeros(len(r_array)) #magma ocean
        M_SM_array = np.zeros(len(r_array)) #solid mantle
        M_atmo_array = np.zeros(len(r_array)) #atmosphere
        M_lost_array = np.zeros(len(r_array)) #space "sink" for lost water
        EL_loss_MO_array = np.zeros(len(r_array))
        DL_loss_MO_array = np.zeros(len(r_array))
        loss_MO_array = np.zeros(len(r_array))
        TOA_flux_MO_array = np.zeros(len(r_array))
        a_orb = a_orb_array[jdx]*1.496e11 #[m]

        # Track MO temperature as a function of r/t.
        T_MO_array = np.zeros(len(t_MO_array))

        # Keep track of the total water inventory, for mass balance purposes throughout.
        M_total = M_init_array[kdx] #[kg]
        
        # Initial concentration; needs to be altered from array if MO begins simulation saturated.
        C_initial = C_0_array[kdx]
        
        # First, need to check if MO holds MORE water than saturation limit; if it does, degas the excess into
        # atmosphere FIRST before running simulations.
        if C_initial > C_sat:
            
            MO_excess = (C_initial-C_sat)*(4.*np.pi*rho*(R_p**3. - R_c**3.))/3.
            
            M_MO_array[0] = M_total - MO_excess
            M_SM_array[0] = 0.
            M_atmo_array[0] = MO_excess
            
            C_initial = (3.*M_MO_array[0])/(4.*np.pi*rho*(R_p**3. - R_c**3.))
            
        else:
        
            # Set initial parameters; all water in MO:
            M_MO_array[0] = M_total
            M_SM_array[0] = 0.
            M_atmo_array[0] = 0.
        
        EL_loss_MO_array[0] = loss_rate_EL_MO(t_MO_array[0], F_XUV(t_MO_array[0], M, a_orb, params1, params2, params3), M_E)
        DL_loss_MO_array[0] = loss_rate_DL_MO(M)
        loss_MO_array[0] = f_loss_MO(t_MO_array[0], M, a_orb, params1, params2, params3)
        TOA_flux_MO_array[0] = S_0(t_MO_array[0], M, a_orb, params1, params2, params3)
        T_MO_array[0] = T_MO(t_MO_array[0], tau_MO_array[jdx], M, a_orb, params1, params2, params3)
            
        f.write(str(t_MO_array[0]) + '\t' + str(r_array[0]) + '\t' + str(T_MO_array[0]) + '\t' + str(M_MO_array[0]) +'\t' + \
                str(M_SM_array[0]) + '\t' + str(M_atmo_array[0]) + '\t' + str(EL_loss_MO_array[0]) + '\t' + \
                str(DL_loss_MO_array[0]) + '\t' +str(loss_MO_array[0]) + '\t' + str(TOA_flux_MO_array[0]) + '\n')

        for idx in range(1,len(r_array)):
    
            EL_loss_MO_array[idx] = loss_rate_EL_MO(t_MO_array[idx], F_XUV(t_MO_array[idx], M, a_orb, params1, params2, params3), M_E)
            DL_loss_MO_array[idx] = loss_rate_DL_MO(M)
            loss_MO_array[idx] = f_loss_MO(t_MO_array[idx], M, a_orb, params1, params2, params3)
            TOA_flux_MO_array[idx] = S_0(t_MO_array[idx], M, a_orb, params1, params2, params3)
            T_MO_array[idx] = T_MO(t_MO_array[idx], tau_MO_array[jdx], M, a_orb, params1, params2, params3)
            
            #MO undersaturated with water, i.e., r <= R_sat
            if r_array[idx] <= R_sat(C_initial):
        
                M_MO_array[idx] = M_MO_unsat(C_initial, r_array[idx])
        
                M_SM_array[idx] = M_SM_unsat(C_initial, r_array[idx])
        
                M_atmo_array[idx] = 0.
        
            # MO saturated/supersaturated with water (r > R_sat); atmosphere forms    
            else: #r_array[idx] > R_sat()
        
                M_MO_array[idx] = M_MO_sat(r_array[idx])
        
                M_SM_array[idx] = M_SM_sat(C_initial, r_array[idx])
        
                M_atmo_array[idx] = M_total - M_SM_sat(C_initial, r_array[idx]) - M_MO_sat(r_array[idx]) - M_lost_array[idx-1]
        
                # Now add loss:
                if M_atmo_array[idx] <= (loss_MO_array[idx]*dt_MO):
            
                    M_lost_array[idx] = M_lost_array[idx-1] + M_atmo_array[idx]
                    M_atmo_array[idx] = 0.  #no water in atmosphere; all lost to space
                        
                else: #M_atmo_array[idx] > (loss_MO_array[idx]*dt_MO)
                    
                    M_atmo_array[idx] = M_atmo_array[idx] - (loss_MO_array[idx]*dt_MO)
                    M_lost_array[idx] = M_lost_array[idx-1] + (loss_MO_array[idx]*dt_MO)

# Comment the below line back in, to write results to file.
            f.write(str(t_MO_array[idx]) + '\t' + str(r_array[idx]) + '\t' + str(T_MO_array[idx]) + '\t' + str(M_MO_array[idx]) +'\t' + \
                str(M_SM_array[idx]) + '\t' + str(M_atmo_array[idx]) + '\t' + str(EL_loss_MO_array[idx]) + '\t' + \
                str(DL_loss_MO_array[idx]) + '\t' +str(loss_MO_array[idx]) + '\t' + str(TOA_flux_MO_array[idx]) + '\n')
                    
        W_m_init_array[kdx, jdx] = M_SM_array[-1]
        W_s_init_array[kdx, jdx] = M_atmo_array[-1]
            
        f.close()

# Define parameters and functions for deep-water cycling.
### The functions in the following cell have been added since the first model iteration.

In [ ]:
# Seafloor pressure
def P(s, M, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return M*g(M)*s/(f_b*4*np.pi*(Rp(M)**2))

# T-dependent hydration depth
def d_h(t, temp, T_surf, x, M, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    return (h(M)**(1.0-3.0*beta))*((temp-T_surf)**(-(1.0+beta)))*\
        (T_serp-T_surf)*((eta(temp,x,params1,params2,params3)*kappa*Ra_c/(alpha*rho_m*g(M)))**beta)

# Radionuclide heating, using 4 common Earth mantle species (i.e., nominal bulk silicate Earth, 21 ppb U)
# (Schaefer & Sasselov 2015, Eqn. (4), with constants as given in Laura's MATLAB code (see refs there))
def Q_sum(t, M, params1, params2, params3):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    
    # Mantle concentration of element by mass (Schubert et al. 2001, Ch. 4)
    Uran = 21e-9; #[U] = 21 ppb
    C_238U = 0.9927 * Uran;
    C_235U = 0.0072 * Uran;
    C_40K = 1.28 * Uran;
    C_232Th = 4.01 * Uran;
    
    # Heat production per unit mass (Schubert et al. 2001, Ch. 4)
    H_238U = 9.37e-5;
    H_235U = 5.69e-4;
    H_232Th = 2.69e-5;
    H_40K = 2.79e-5;
    
    # Decay constants
    lam_238U = 0.155e-9;
    lam_235U = 0.985e-9;
    lam_232Th = 0.0495e-9;
    lam_40K = 0.555e-9;
    
    return rho_m*(C_238U*H_238U*np.exp(lam_238U*(4.6e9-t)) + C_235U*H_235U*np.exp(lam_235U*(4.6e9-t)) + \
            C_232Th*H_232Th*np.exp(lam_232Th*(4.6e9-t)) + C_40K*H_40K*np.exp(lam_40K*(4.6e9-t)))

In [ ]:
# Things that need to be calculated with constants and above functions:
# NOTE: If dimensions are being used anyway, many of these are redundant/won't be used in the code.
omg_Etwid = num_oceans*omega_E/omega_0
omegatwid = omg_Etwid/(f_btwid)
T_stwid = T_s/T_ref
T_mtwid = T_ref*R_g/E_a
T_liqtwid = T_liq_dry/T_ref
T_soltwid = T_sol_dry/T_ref
Ktwid = K_cnst/T_ref
Xtwid = x_h*rho_c*chi*d_hE*f_M/(rho_m*d_melt*f_degasE*omega_0*f_btwid)
Pi = (rho_m*d_melt*chi_d*(omega_0*f_btwid/f_M)*((T_liqtwid-T_soltwid)**-theta))
lambdatwid = Ktwid*(omega_0*f_btwid/f_M)**(gamma)
eta_scale = (np.exp(lnf_w(5.8e-4,params1,params2,params3)))**(-r_fug)
eta_0 = 1.0e21/eta_scale
#f_w = np.exp(lnf_w(1.,params1,params2,params3))
f_wtwid_min = 1.0e-5 #CONSERVATIVELY CHOSEN FOR NOW
E = rho_m*d_melt*f_degasE*omega_0*f_btwid/f_M
#tau_heat = Q_0/(rho_m*c_p*T_ref)

# 4) Calculated parameters
params4 = [omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E]

# 1-D Atmosphere & Loss Functions

In [ ]:
# Other variables/functions, required for the adiabats and water saturation pressure.

P_0_star = 1.4e11 #[Pa]
l_c_NJ = 43655 #[J/mol] THIS VALUE IS USED BY NAKAJIMA+ 1992
l_c = 2441.7e3 # [J/kg] (https://www.engineeringtoolbox.com/water-properties-d_1573.html)
R_gas_NJ = 8.314 #[J/mol/K] THIS VALUE IS USED BY NAKAJIMA+ 1992
R_gas = 8314. #[J/kg/K]

# Saturation water vapour pressure
def P_star(T): 
    
    return P_0_star*np.exp(-l_c_NJ/(R_gas_NJ*T))

In [ ]:
# Use water condensation insolation threshold of 325 W/m^2 (Turbet+ 2021) to calculate surface temperature.

def T_surf_OLR(t, M, alb, a_orb, params1, params2, params3):
    
    if S_0(t, M, a_orb, params1, params2, params3)/4. > (325./(1.-alb)): #runaway greenhouse
                
        return 1800. #~maximum T_surf corresponding to given OLR
    
    else: #not in runaway greenhouse
        
        return 293.15 # 20 C

In [ ]:
# Function to determine total mass of water in atmosphere, given T_surf, surface water inventory, albedo.
def f_M_H2O(t, W_s, T_surf, alb, M, m, a_orb, params1, params2, params3):
    
    # Runaway greenhouse limit; divide by 4 because planet is a sphere
    # All surface water in atmosphere
    if S_0(t, M, a_orb, params1, params2, params3)/4. > (325./(1.-alb)):
        
        return W_s #[kg]
        
    else:
        
        return (P_star(T_surf)/g(M))*4.*np.pi*(Rp(M)**2.) #[kg]
    
def f_P_surf(t, W_s, T_surf, alb, M, m, a_orb, params1, params2, params3):
    
    # Runaway greenhouse limit; divide by 4 because planet is a sphere
    # All surface water in atmosphere
    if S_0(t, M, a_orb, params1, params2, params3)/4. > (325./(1.-alb)):
        
        return W_s*g(M)/(4.*np.pi*(Rp(M)**2.))
    
    else:
        
        return P_star(T_surf)

### Energy-limited & diffusion-limited loss, from Luger & Barnes (2015)

In [ ]:
# Energy-limited escape rate
def loss_rate_EL(t, F_XUV, R_XUV, M): #[kg/s]
    
    # R_XUV = XUV deposition radius; LB15 set R_XUV = R_p for simplicity
    # M = planetary mass
    
    G = 6.67e-11
    eps_XUV = 0.1 #nominal; LB15 test 0.15-0.30
    
    return eps_XUV*np.pi*F_XUV*Rp(M)*(R_XUV**2.)/(G*M)

# Diffusion-limited escape rate
def loss_rate_DL(t, T_surf, T_strat, M):
    
    g_p = g(M)*100. #[cm s^-2], to match the units of b
    k_B = 1.38e-23 #[m^2 kg s^-2 K^-1]; Boltzmann constant
    
    # Mole fractions of H & O (assuming all H2O is dissociated)
    X_H = 2./3.
    X_O = 1./3.
    
    m_H = 1.66e-27 #[kg]; mass of H atom
    m_O = 16.*m_H
        
    # Just take the thermospheric temperature as 400 K at all times, for now.
    b = 4.8e17*(400.**0.75) #[cm^-1 s^-1]
        
    return m_H*(np.pi*(Rp(M)**2.)*b*g_p*(m_O-m_H))/(k_B*400.*(1.+(X_O/X_H))) 

# Cycling Functions

In [ ]:
# Thermal evolution of mantle temperature
def f_delta_temp(t, temp, W_m, W_s, T_surf, M, params1, params2, params3, params4):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E = params4
    
    x = W_m/(f_M*M)
    s = W_s/M
    
    return (1./(rho_m*c_p))*((Q_sum(t, M, params1, params2, params3)) - \
        (k*A(M)*(temp-T_surf)/(h(M)*V(M)))*(Ra(t, temp, x, T_surf, M, params1, params2, params3)/Ra_c)**beta)

# Degassing rate (using 1% melt approximation of Komaceck & Abbot 2016)
def f_degas(t, temp, W_m, W_s, T_surf, M, params1, params2, params3, params4):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E = params4
    
    x = W_m/(f_M*M)
    s = W_s/M
    
    # Mantle temperature check
    if temp > T_sol_dry-K_cnst*x**gamma: 
    
        # Piecewise limit from CA2014
        if x > 0. and s > 0. and f_degasE*(P(s, M, params1, params2, params3)/P_E)**(-mu) < 1.:
            return L_MOR(M)*S(t, temp, x, T_surf, M, params1, params2, params3)*(x*rho_m*d_melt*\
                (f_degasE*(P(s, M, params1, params2, params3)/P_E)**(-mu))*\
                ((temp-(T_sol_dry-K_cnst*x**gamma))/(T_liq_dry-T_sol_dry))**theta)
    
        elif x > 0. and s > 0. and f_degasE*(P(s, M, params1, params2, params3)/P_E)**(-mu) >= 1.:
            return L_MOR(M)*S(t, temp, x, T_surf, M, params1, params2, params3)*(x*rho_m*d_melt*\
                (1.)*((temp-(T_sol_dry-K_cnst*x**gamma))/(T_liq_dry-T_sol_dry))**theta)
    
        elif x > 0. and s <= 0.:
            return L_MOR(M)*S(t, temp, x, T_surf, M, params1, params2, params3)*(x*rho_m*d_melt*\
                (1.)*((temp-(T_sol_dry-K_cnst*x**gamma))/(T_liq_dry-T_sol_dry))**theta)
    
        else: # x <= 0.; no degassing
            return 0.
        
    else: # temp <= T_sol_dry-K_cnst*x**gamma; no degassing
        return 0.
    
# Regassing rate (including regassing check of CA14 in T-dependent regassing of SS15)
def f_regas(t, temp, W_m, W_s, T_surf, alb, M, a_orb, params1, params2, params3, params4):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E = params4
    
    x = W_m/(f_M*M)
    s = W_s/M
    
    if S_0(t, M, a_orb, params1, params2, params3)/4. > (325./(1.-alb)):
        
        # Runaway greenhouse; all water in atmosphere so NO regassing!!!
        return 0.
        
    else:
    
        # Piecewise limit from CA2014
        if x > 0. and s > 0. and d_h(t, temp, T_surf, x, M, params1, params2, params3)*(P(s, M, params1, params2, params3)/P_E)**sigma < d_b:
            return L_MOR(M)*S(t, temp, x, T_surf, M, params1, params2, params3)*(x_h*rho_c*chi_r*\
                d_h(t, temp, T_surf, x, M, params1, params2, params3)*(P(s, M, params1, params2, params3)/P_E)**sigma)
    
        elif x > 0. and s > 0. and d_h(t, temp, T_surf, x, M, params1, params2, params3)*(P(s, M, params1, params2, params3)/P_E)**sigma >= d_b:
            return L_MOR(M)*S(t, temp, x, T_surf, M, params1, params2, params3)*(x_h*rho_c*chi_r*d_b)
    
        elif x <= 0. and s > 0.: # not a big fan of this condition choice (constant spreading rate)
            return L_MOR(M)*(0.1/year)*(x_h*rho_c*chi_r*d_b)
    
        elif s <= 0.: # no regassing
            return 0.
    
# Water loss rate; ENERGY-LIMITED OR DIFFUSION-LIMITED ESCAPE (Luger & Barnes 2015)
def f_loss(t, temp, W_m, W_s, T_surf, T_strat, alb, dt, M, a_orb, params1, params2, params3, params4):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E = params4
    
    x = W_m/(f_M*M)
    s = W_s/M
    
    # Runaway greenhouse limit; divide by 4 because planet is a sphere
    if S_0(t, M, a_orb, params1, params2, params3)/4. > (325./(1.-alb)):
        
        # Energy-limited loss; set R_XUV = R_p, for simplicity
        M_loss = loss_rate_EL(t, F_XUV(t, M, a_orb, params1, params2, params3), Rp(M), M) #[kg/s]
        
    else:
           
        # Take the lower value of EL and DL loss rates. 
        M_loss = np.minimum(loss_rate_EL(t, F_XUV(t, M, a_orb, params1, params2, params3), Rp(M), M), \
                        loss_rate_DL(t, T_surf, T_strat, M)) #[kg/s]
        
    # For our purposes, replenishment time above exobase is ~instantaneous; therefore, we can take water directly
    # from the surface reservoir, for now.
    # Piecewise def'n from me, so you don't lose more than on the surface
    if W_s == 0.:
        return 0.
    elif M_loss < W_s/dt:
        return M_loss
    else:
        return W_s/dt
    
    #return 0. #no loss

In [ ]:
# Mantle water evolution over time
def f_delta_W_m(t, temp, W_m, W_s, T_surf, alb, dt, M, a_orb, params1, params2, params3, params4):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E = params4
    
    x = W_m/(f_M*M)
    s = W_s/M
    
    d_W_m = f_regas(t, temp, W_m, W_s, T_surf, alb, M, a_orb, params1, params2, params3, params4) - \
            f_degas(t, temp, W_m, W_s, T_surf, M, params1, params2, params3, params4)
    
    # Mantle water capacity check
    if W_m + (d_W_m*dt) <= 12.0*1.4e21:
        return d_W_m
    else:
        return ((12.0*1.4e21) - W_m)/dt

# Surface water evolution over time
def f_delta_W_s(t, temp, W_m, W_s, T_surf, T_strat, alb, dt, M, a_orb, params1, params2, params3, params4):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E = params4
    
    x = W_m/(f_M*M)
    s = W_s/M
    
    d_W_m = f_regas(t, temp, W_m, W_s, T_surf, alb, M, a_orb, params1, params2, params3, params4) - \
            f_degas(t, temp, W_m, W_s, T_surf, M, params1, params2, params3, params4)
    
    # Mantle water capacity check
    if W_m + (d_W_m*dt) <= 12.0*1.4e21:
        return f_degas(t, temp, W_m, W_s, T_surf, M, params1, params2, params3, params4) - \
            f_regas(t, temp, W_m, W_s, T_surf, alb, M, a_orb, params1, params2, params3, params4) - \
            f_loss(t, temp, W_m, W_s, T_surf, T_strat, alb, dt, M, a_orb, params1, params2, params3, params4)
    else:
        return -(((12.0*1.4e21) - W_m)/dt) - \
            f_regas(t, temp, W_m, W_s, T_surf, alb, M, a_orb, params1, params2, params3, params4) - \
            f_loss(t, temp, W_m, W_s, T_surf, T_strat, alb, dt, M, a_orb, params1, params2, params3, params4)

# Coupled cycling equations, to be integrated
def f_cycling(t, z, T_surf, T_strat, alb, dt, M, a_orb, params1, params2, params3, params4):
    
    M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
    x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
        T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
    omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
    omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
           eta_scale, eta_0, f_w, f_wtwid_min, E = params4
    
    temp = z[0]
    W_m = z[1]
    W_s = z[2]
    
    return [f_delta_temp(t, temp, W_m, W_s, T_surf, M, params1, params2, params3, params4),\
                f_delta_W_m(t, temp, W_m, W_s, T_surf, alb, dt, M, a_orb, params1, params2, params3, params4),\
                f_delta_W_s(t, temp, W_m, W_s, T_surf, T_strat, alb, dt, M, a_orb, params1, params2, params3, params4)]

In [ ]:
## Orbital distances.
# Read in Kopparapu+(2013) HZ data, for different stellar hosts.

data = np.loadtxt('HZs.dat', skiprows=2)
T_eff_kopp = data[:,0] #[K]
# All below values give S_eff -- see Eqn.(2) of Kopparapu+(2013)
RV_kopp = data[:,1] #Recent Venus
RG_kopp = data[:,2] #Runaway Greenhouse
MG_kopp = data[:,3] #Maximum Greenhouse
EM_kopp = data[:,4] #Early Mars

def f_a_orb(t, M, T_eff_star, Lbol_Ls_star, params1, params2, params3):
    
    S_eff_RV = np.interp(T_eff_star, T_eff_kopp, RV_kopp)
    S_eff_RG = np.interp(T_eff_star, T_eff_kopp, RG_kopp)
    S_eff_MG = np.interp(T_eff_star, T_eff_kopp, MG_kopp)
    S_eff_EM = np.interp(T_eff_star, T_eff_kopp, EM_kopp)
    
    a_RV = (Lbol_Ls_star/S_eff_RV)**0.5 #[AU]
    a_RG = (Lbol_Ls_star/S_eff_RG)**0.5 #[AU]
    a_MG = (Lbol_Ls_star/S_eff_MG)**0.5 #[AU]
    a_EM = (Lbol_Ls_star/S_eff_EM)**0.5 #[AU]
    
    return a_RV, a_RG, a_MG, a_EM

# Time-dependent Cycling & Loss Simulations

In [ ]:
# Need various values of initial water inventories -- USE OUTPUT FROM ABOVE MO MODEL RUNS.

#tau_MO = tau_RG; tau_RG_array defined in MO module of notebook.

#M_init_array is the TOTAL initial water inventory, which starts in the MO.
M_init_file = np.array([2., 4., 6., 8.]) #[Earth Oceans]
#W_m_init_array is the mantle water
#W_s_init_array is the surface water

# Also need orbital distances for the given host star
star_age, Lbol, Lbol_Ls_star, T_eff_star = hoststar(hostfile)
    
# Roughly find limits at t = 4.5 Gyr (~age of Earth)
T_eff_star_45 = np.interp(4.5e9*year, star_age, T_eff_star)
Lbol_Ls_star_45 = np.interp(4.5e9*year, star_age, Lbol_Ls_star)
        
a_RV, a_RG, a_MG, a_EM = f_a_orb(4.5e9*year, M_E, T_eff_star_45, Lbol_Ls_star_45, params1, params2, params3)
    
# For now, take runaway greenhouse and maximum greenhouse limits, and then the middle between them.
a_orb_array = np.array([a_RG, (a_RG+a_MG)/2., a_MG])
a_orb_labels = ['Runaway_GH', 'Middle', 'Maximum_GH']

In [ ]:
print(a_orb_array)

## Change the cell below from MARKDOWN to CODE if running simulations.

In [ ]:
# Loop through the above arrays, by solidification timescale and water inventory.
    
for kdx in range(0, len(M_init_array)):
        
    for jdx in range(0, len(a_orb_array)):
        
        # Set initial conditions.
        t0 = tau_MO_array[jdx] #START AT END OF MO PHASE
        z0 = [3000., W_m_init_array[kdx, jdx], W_s_init_array[kdx, jdx]] #[K] [kg] [kg]
        M = M_E
        # Atmosphere initial conditions.
        m = m_H2O #molecular mass of water 
        a_orb = a_orb_array[jdx]*1.496e11 #[m]
        alb = 0.3 #0.1 #albedo
        T_surf_0 = T_surf_OLR(t0, M, alb, a_orb, params1, params2, params3)
        P_surf_0 = f_P_surf(t0, z0[2], T_surf_0, alb, M, m, a_orb, params1, params2, params3)
        T_strat_0 = T_strat(t0, M, a_orb, params1, params2, params3)
        # Define max time, timestep, arrays to be filled within the integration loop.
        t1 = 5.0e9*year #5 Gyr
        dt = 2.0e4*year #20,000 yr per timestep

        count_array = np.zeros(int((t1-t0)/dt)+1) #IF LIMITED BY AMOUNT OF WATER, SET = 1; OTHERWISE, LEAVE AS 0

        # Set up function to be integrated.
        r = ode(f_cycling).set_integrator('vode')
        r.set_initial_value(z0, t0).set_f_params(T_surf_0,T_strat_0,alb,dt,M,a_orb,params1,params2,params3,params4)

        # Define arrays to be filled within the integration loop.
        t_array = np.zeros(int((t1-t0)/dt)+1)
        # Cycling
        T_array = np.zeros(int((t1-t0)/dt)+1)
        W_m_array = np.zeros(int((t1-t0)/dt)+1)
        W_s_array = np.zeros(int((t1-t0)/dt)+1)
        regas_array = np.zeros(int((t1-t0)/dt)+1)
        degas_array = np.zeros(int((t1-t0)/dt)+1)
        # Atmosphere
        T_surf_array = np.zeros(int((t1-t0)/dt)+1)
        T_strat_array = np.zeros(int((t1-t0)/dt)+1)
        TOA_flux_array = np.zeros(int((t1-t0)/dt)+1)
        loss_array = np.zeros(int((t1-t0)/dt)+1)
        EL_array = np.zeros(int((t1-t0)/dt)+1)
        DL_array = np.zeros(int((t1-t0)/dt)+1)
        M_H2O_array = np.zeros(int((t1-t0)/dt)+1)
        P_surf_array = np.zeros(int((t1-t0)/dt)+1)
        
        # Write results to file, for plotting later.
        save_file = 'model2_combined_MO_cycling_loss_M_init-' + str(M_init_file[kdx]) + 'TO_' + str(a_orb_labels[jdx]) + '.txt'

        filename = os.path.join(save_path, save_file)
        f = open(filename, 'w')
        
        # Initial values in the arrays.
        t_array[0] = t0
        # Cycling
        T_array[0] = z0[0]
        W_m_array[0] = z0[1]
        W_s_array[0] = z0[2]
        regas_array[0] = f_regas(t0, z0[0], z0[1], z0[2], T_surf_0, alb, M, a_orb, params1, params2, params3, params4)
        degas_array[0] = f_degas(t0, z0[0], z0[1], z0[2], T_surf_0, M, params1, params2, params3, params4)
        # Atmosphere
        T_surf_array[0] = T_surf_0
        T_strat_array[0] = T_strat_0
        TOA_flux_array[0] = S_0(t0, M, a_orb, params1, params2, params3)
        loss_array[0] = f_loss(t0, z0[0], z0[1], z0[2], T_surf_0, T_strat_0, alb, dt, M, a_orb, params1, params2, params3, params4)
        EL_array[0] = loss_rate_EL(t0, F_XUV(t0, M, a_orb, params1, params2, params3), Rp(M), M) #[kg/s]
        DL_array[0] = loss_rate_DL(t0, T_surf_0, T_strat_0, M) #[kg/s]
        M_H2O_array[0] = f_M_H2O(t0, z0[2], T_surf_0, alb, M, m, a_orb, params1, params2, params3)
        P_surf_array[0] = P_surf_0
        
        # Write initial values to file.
        f.write(str(t_array[0]) + '\t' + str(T_array[0]) + '\t' + str(W_m_array[0]) +'\t' + str(W_s_array[0]) + \
                '\t' + str(degas_array[0]) + '\t' + str(regas_array[0]) + '\t' + str(T_surf_array[0]) + '\t' + \
                str(T_strat_array[0]) + '\t' + str(TOA_flux_array[0]) + '\t' + str(loss_array[0]) + '\t' + \
                str(EL_array[0]) + '\t' + str(DL_array[0]) + '\t' + str(M_H2O_array[0]) + '\t' + \
                str(P_surf_array[0]) + '\n')

        # Integrate the above function.
        idx = 1
        start_time = time.time()
        for idx in range(1,len(t_array)):
    
            if r.successful() == True:
                r.integrate(r.t+dt)
                t_array[idx] = r.t
                T_array[idx] = r.y[0]
                W_m_array[idx] = r.y[1]
                W_s_array[idx] = r.y[2]
        
                ########
                # Runaway greenhouse limit; divide by 4 because planet is a sphere
                if S_0(r.t, M, a_orb, params1, params2, params3)/4. > (325./(1.-alb)):
        
                    # Energy-limited loss; set R_XUV = R_p, for simplicity
                    M_loss = loss_rate_EL(r.t, F_XUV(r.t, M, a_orb, params1, params2, params3), Rp(M), M) #[kg/s]
        
                else:
           
                    # Take the lower value of EL and DL loss rates. 
                    M_loss = np.minimum(loss_rate_EL(r.t, F_XUV(r.t, M, a_orb, params1, params2, params3), Rp(M), M), \
                                    loss_rate_DL(r.t, T_surf_array[idx], T_strat_array[idx], M)) #[kg/s]
        
                # Piecewise def'n from me, so you don't lose more than on the surface
                if M_loss*dt > W_s_array[idx]:
                    count_array[idx] = 1
                ########
        
                T_surf_array[idx] = T_surf_OLR(r.t, M, alb, a_orb, params1, params2, params3)
                P_surf_array[idx] = f_P_surf(r.t, r.y[2], T_surf_array[idx], alb, M, m, a_orb, params1, params2, params3)
                T_strat_array[idx] = T_strat(r.t, M, a_orb, params1, params2, params3)
                TOA_flux_array[idx] = S_0(r.t, M, a_orb, params1, params2, params3)
        
                regas_array[idx] = f_regas(r.t, r.y[0], r.y[1], r.y[2], T_surf_array[idx], alb, M, a_orb, params1, params2, params3, params4)
                degas_array[idx] = f_degas(r.t, r.y[0], r.y[1], r.y[2], T_surf_array[idx], M, params1, params2, params3, params4)
                loss_array[idx] = f_loss(r.t, r.y[0], r.y[1], r.y[2], T_surf_array[idx], T_strat_array[idx], alb, dt, M, a_orb, params1, params2, params3, params4)
                EL_array[idx] = loss_rate_EL(r.t, F_XUV(r.t, M, a_orb, params1, params2, params3), Rp(M), M) #[kg/s]
                DL_array[idx] = loss_rate_DL(r.t, T_surf_array[idx], T_strat_array[idx], M) #[kg/s]
                M_H2O_array[idx] =  f_M_H2O(r.t, r.y[2], T_surf_array[idx], alb, M, m, a_orb, params1, params2, params3)
        
                # Write current values to file.
                f.write(str(t_array[idx]) + '\t' + str(T_array[idx]) + '\t' + str(W_m_array[idx]) +'\t' + str(W_s_array[idx]) + \
                    '\t' + str(degas_array[idx]) + '\t' + str(regas_array[idx]) + '\t' + str(T_surf_array[idx]) + '\t' + \
                    str(T_strat_array[idx]) + '\t' + str(TOA_flux_array[idx]) + '\t' + str(loss_array[idx]) + '\t' + \
                    str(EL_array[idx]) + '\t' + str(DL_array[idx]) + '\t' + str(M_H2O_array[idx]) + '\t' + \
                    str(P_surf_array[idx]) + '\n')
                
            elif r.successful() == False:
                M_E, R_E, d_hE, rho_c, rho_m, f_degasE, G, P_E, year, omega_E, x_E, E_a, R_g, T_liq_dry, T_sol_dry = params1
                x_h, chi, d_melt, f_M, f_b, f_btwid, alpha, Ra_c, kappa, T_ref, k, beta, c_p, Q_0, T_s, theta, \
                    T_serp, K_cnst, gamma, r_fug, d_b, f_w_min = params2
                omega_0, num_oceans, sigma, mu, t_loss, loss_factor, chi_d, chi_r, dt_nom = params3
                omg_Etwid, omegatwid, T_stwid, T_mtwid, T_liqtwid, T_soltwid, Ktwid, Xtwid, Pi, lambdatwid, \
                    eta_scale, eta_0, f_w, f_wtwid_min, E = params4
       
                t_array[idx] = t_array[idx-1] + dt
                T_array[idx] = T_array[idx-1] + \
                    f_delta_temp(t_array[idx-1], T_array[idx-1], W_m_array[idx-1], W_s_array[idx-1], T_surf_array[idx-1], M, params1, params2, params3, params4)*dt
                W_m_array[idx] = W_m_array[idx-1] + \
                    f_delta_W_m(t_array[idx-1], T_array[idx-1], W_m_array[idx-1], W_s_array[idx-1], T_surf_array[idx-1], alb, dt, M, a_orb, params1, params2, params3, params4)*dt
                W_s_array[idx] = W_s_array[idx-1] + \
                    f_delta_W_s(t_array[idx-1], T_array[idx-1], W_m_array[idx-1], W_s_array[idx-1], T_surf_array[idx-1], T_strat_array[idx-1], alb, dt, M, a_orb, params1, params2, params3, params4)*dt

                ########
                # Runaway greenhouse limit; divide by 4 because planet is a sphere
                if S_0(t_array[idx], M, a_orb, params1, params2, params3)/4. > (325./(1.-alb)):
        
                    # Energy-limited loss; set R_XUV = R_p, for simplicity
                    M_loss = loss_rate_EL(t_array[idx], F_XUV(t_array[idx], M, a_orb, params1, params2, params3), Rp(M), M) #[kg/s]
        
                else:
           
                    # Take the lower value of EL and DL loss rates. 
                    M_loss = np.minimum(loss_rate_EL(t_array[idx], F_XUV(t_array[idx], M, a_orb, params1, params2, params3), Rp(M), M), \
                                        loss_rate_DL(t_array[idx], T_surf_array[idx], T_strat_array[idx], M)) #[kg/s]
        
                # Piecewise def'n from me, so you don't lose more than on the surface
                if M_loss*dt > W_s_array[idx]:
                    count_array[idx] = 1
                ########
                
                T_surf_array[idx] = T_surf_OLR(t_array[idx], M, alb, a_orb, params1, params2, params3)
                P_surf_array[idx] = f_P_surf(t_array[idx], W_s_array[idx], T_surf_array[idx], alb, M, m, a_orb, params1, params2, params3)
                T_strat_array[idx] = T_strat(t_array[idx], M, a_orb, params1, params2, params3)
                TOA_flux_array[idx] = S_0(t_array[idx], M, a_orb, params1, params2, params3)
        
                regas_array[idx] = f_regas(t_array[idx], T_array[idx-1], W_m_array[idx], W_s_array[idx], T_surf_array[idx], alb, M, a_orb, params1, params2, params3, params4)
                degas_array[idx] = f_degas(t_array[idx], T_array[idx-1], W_m_array[idx], W_s_array[idx], T_surf_array[idx], M, params1, params2, params3, params4)
                loss_array[idx] = f_loss(t_array[idx], T_array[idx-1], W_m_array[idx], W_s_array[idx], T_surf_array[idx], T_strat_array[idx], alb, dt, M, a_orb, params1, params2, params3, params4)
                EL_array[idx] = loss_rate_EL(t_array[idx], F_XUV(t_array[idx], M, a_orb, params1, params2, params3), Rp(M), M) #[kg/s]
                DL_array[idx] = loss_rate_DL(t_array[idx], T_surf_array[idx], T_strat_array[idx], M) #[kg/s]
                M_H2O_array[idx] =  f_M_H2O(t_array[idx], W_s_array[idx], T_surf_array[idx], alb, M, m, a_orb, params1, params2, params3)
                
                # Write current values to file.
                f.write(str(t_array[idx]) + '\t' + str(T_array[idx]) + '\t' + str(W_m_array[idx]) +'\t' + str(W_s_array[idx]) + \
                    '\t' + str(degas_array[idx]) + '\t' + str(regas_array[idx]) + '\t' + str(T_surf_array[idx]) + '\t' + \
                    str(T_strat_array[idx]) + '\t' + str(TOA_flux_array[idx]) + '\t' + str(loss_array[idx]) + '\t' + \
                    str(EL_array[idx]) + '\t' + str(DL_array[idx]) + '\t' + str(M_H2O_array[idx]) + '\t' + \
                    str(P_surf_array[idx]) + '\n')

        if W_m_array[-1]+W_s_array[-1] <= 1.29e16:
            print('Planet Desiccated')
            
        f.close()
    
        end_time = time.time()
        print(end_time-start_time, ' s')
        print('Escape limited by surface water: ', np.sum(count_array), 'of ', (int(t1/dt)+1), 'runs.')

# Read in the simulation results from files. 
## Read in all files for host star, and save initial/final water inventories.
## Separate solidification timescales into separate cells, for read-in and plotting. 

In [ ]:
# Read in MO results (pre-cycling).

#tolerance = 1.29e16 #[kg]; if below this, set water = 0 --> DESICCATED

# Initial water content in each reservoir
M_MO_initial_all = np.zeros((len(M_init_array), len(a_orb_array)))
M_SM_initial_all = np.zeros((len(M_init_array), len(a_orb_array)))
M_atmo_initial_all = np.zeros((len(M_init_array), len(a_orb_array)))

# Final water content in each reservoir
M_MO_final_all = np.zeros((len(M_init_array), len(a_orb_array)))
M_SM_final_all = np.zeros((len(M_init_array), len(a_orb_array)))
M_atmo_final_all = np.zeros((len(M_init_array), len(a_orb_array)))

# Initial and final mantle temperatures, if needed.
T_MO_initial_all = np.zeros((len(M_init_array), len(a_orb_array)))
T_MO_final_all = np.zeros((len(M_init_array), len(a_orb_array)))

# How many reach exactly zero in one/both reservoir/s at the end?
#W_m_zero_count_MO_all = 0.
#W_s_zero_count_MO_all = 0.
#both_zero_count_MO_all = 0.

for jdx in range(0, len(a_orb_array)):
    
    # Need length of data for the following arrays.
    save_file = 'model2_MO_only_loss_M_init-2.0TO_' + str(a_orb_labels[jdx]) + '.txt'    
    filename = os.path.join(save_path, save_file)
    tmp_data = np.loadtxt(filename)

    # Time array; same for all with the same orbital distance.
    t_MO_array = tmp_data[:, 0]

    # Overall evolution path for each reservoir & mantle temperature & surface temperature.
    M_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
    M_SM_paths = np.zeros((len(M_init_array),len(tmp_data)))
    M_atmo_paths = np.zeros((len(M_init_array),len(tmp_data)))
    r_sol_paths = np.zeros((len(M_init_array),len(tmp_data)))
    T_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
    TOA_flux_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
    #TOA_flux_paths = np.zeros((len(tau_MO_array), len(M_init_array),len(tmp_data)))

    # Overall evolution path for degassing and regassing rates.
    loss_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
    EL_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
    DL_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
    
    for kdx in range(0, len(M_init_array)):
                
        # Read in data files for each model, and save them as the evolutionary path. 
        save_file = 'model2_MO_only_loss_M_init-' + str(M_init_file[kdx]) + 'TO_' + str(a_orb_labels[jdx]) + '.txt'
            
        print(save_file)
        filename = os.path.join(save_path, save_file)
        tmp_data = np.loadtxt(filename)
        r_sol_paths[kdx,:] = tmp_data[:,1]
        T_MO_paths[kdx:] = tmp_data[:,2]
        M_MO_paths[kdx,:] = tmp_data[:,3]
        M_SM_paths[kdx,:] = tmp_data[:,4]
        M_atmo_paths[kdx,:] = tmp_data[:,5]
        EL_MO_paths[kdx,:] = tmp_data[:,6]
        DL_MO_paths[kdx,:] = tmp_data[:,7]
        loss_MO_paths[kdx,:] = tmp_data[:,8]
        TOA_flux_MO_paths[kdx,:] = tmp_data[:,9]

        # Save initial and final values to corresponding arrays.
        M_MO_initial_all[kdx, jdx] = M_MO_paths[kdx,0]
        M_SM_initial_all[kdx, jdx] = M_SM_paths[kdx,0]
        M_atmo_initial_all[kdx, jdx] = M_atmo_paths[kdx,0]
    
        T_MO_initial_all[kdx, jdx] = T_MO_paths[kdx,0]
        T_MO_final_all[kdx, jdx] = T_MO_paths[kdx,-1]
    
        # Check for zeros.     
        #if W_m_paths[kdx,ldx,-1] <= tolerance and W_s_paths[kdx,ldx,-1] <= tolerance:
         #    both_zero_count_all = both_zero_count_all + 1.
            #W_m_final[jdx,kdx] = 0.
            #W_s_final[jdx,kdx] = 0.
        #elif W_m_paths[kdx,ldx,-1] <= tolerance:
        #    W_m_zero_count_all = W_m_zero_count_all + 1.
            #W_m_final[jdx,kdx] = 0.
            #W_s_final[jdx,kdx] = W_s_paths[jdx,kdx,-1]
        #elif W_s_paths[kdx,ldx,-1] <= tolerance:
        #    W_s_zero_count_all = W_s_zero_count_all + 1.
            #W_s_final[jdx,kdx] = 0.
            #W_m_final[jdx,kdx] = W_m_paths[jdx,kdx,-1]
        #else: #neither is zero; save normally
        #    W_m_final_all[jdx, kdx, ldx] = W_m_paths[kdx,ldx,-1]
        #    W_s_final_all[jdx, kdx, ldx] = W_s_paths[kdx,ldx,-1]
        
        M_MO_final_all[kdx, jdx] = M_MO_paths[kdx,-1]
        M_SM_final_all[kdx, jdx] = M_SM_paths[kdx,-1]     
        M_atmo_final_all[kdx, jdx] = M_atmo_paths[kdx,-1]
            
#print('Zero water: ', 'W_m: ', W_m_zero_count_all, 'W_s: ', W_s_zero_count_all, 'both: ', both_zero_count_all)

In [ ]:
## Read in cycling results (post-MO).

tolerance = 1.29e16 #[kg]; if below this, set water = 0 --> DESICCATED

# Initial water content in each reservoir
W_m_initial_all = np.zeros((len(M_init_array), len(a_orb_array)))
W_s_initial_all = np.zeros((len(M_init_array), len(a_orb_array)))

# Final water content in each reservoir
W_m_final_all = np.zeros((len(M_init_array), len(a_orb_array)))
W_s_final_all = np.zeros((len(M_init_array), len(a_orb_array)))

# Initial and final mantle temperatures, if needed.
T_initial_all = np.zeros((len(M_init_array), len(a_orb_array)))
T_final_all = np.zeros((len(M_init_array), len(a_orb_array)))

# How many reach exactly zero in one/both reservoir/s at the end?
W_m_zero_count_all = 0.
W_s_zero_count_all = 0.
both_zero_count_all = 0.

for jdx in range(0, len(a_orb_array)):
    
    # Need length of data for the following arrays.
    save_file = 'model2_combined_MO_cycling_loss_M_init-2.0TO_' + str(a_orb_labels[jdx]) + '.txt'   
    filename = os.path.join(save_path, save_file)
    tmp_data = np.loadtxt(filename)

    # Time array; same for all.
    t_array = tmp_data[:, 0]

    # Overall evolution path for each reservoir & mantle temperature & surface temperature.
    W_m_paths = np.zeros((len(M_init_array),len(tmp_data)))
    W_s_paths = np.zeros((len(M_init_array),len(tmp_data)))
    T_paths = np.zeros((len(M_init_array),len(tmp_data)))
    T_surf_paths = np.zeros((len(M_init_array),len(tmp_data)))
    TOA_flux_paths = np.zeros((len(M_init_array),len(tmp_data)))
    #TOA_flux_paths = np.zeros((len(tau_MO_array), len(M_init_array),len(tmp_data)))

    # Overall evolution path for degassing and regassing rates.
    degas_paths = np.zeros((len(M_init_array),len(tmp_data)))
    regas_paths = np.zeros((len(M_init_array),len(tmp_data)))
    loss_paths = np.zeros((len(M_init_array),len(tmp_data)))
    EL_paths = np.zeros((len(M_init_array),len(tmp_data)))
    DL_paths = np.zeros((len(M_init_array),len(tmp_data)))
    
    for kdx in range(0, len(M_init_array)):
        
        # Read in data files for each model, and save them as the evolutionary path. 
        save_file = 'model2_combined_MO_cycling_loss_M_init-' + str(M_init_file[kdx]) + 'TO_' + str(a_orb_labels[jdx]) + '.txt'
        print(save_file)
        filename = os.path.join(save_path, save_file)
        tmp_data = np.loadtxt(filename)
        T_paths[kdx,:] = tmp_data[:,1]
        W_m_paths[kdx,:] = tmp_data[:,2]
        W_s_paths[kdx,:] = tmp_data[:,3]
        degas_paths[kdx,:] = tmp_data[:,4]
        regas_paths[kdx,:] = tmp_data[:,5]
        T_surf_paths[kdx,:] = tmp_data[:,6]
        TOA_flux_paths[kdx,:] = tmp_data[:,8]
        loss_paths[kdx,:] = tmp_data[:,9]
        EL_paths[kdx,:] = tmp_data[:,10]
        DL_paths[kdx,:] = tmp_data[:,11]
        #DL_paths[jdx,kdx,:] = tmp_data[:,11]

        # Save initial and final values to corresponding arrays.
        W_m_initial_all[kdx, jdx] = W_m_paths[kdx,0]
        W_s_initial_all[kdx, jdx] = W_s_paths[kdx,0]
    
        T_initial_all[kdx, jdx] = T_paths[kdx,0]
        T_final_all[kdx, jdx] = T_paths[kdx,-1]
    
        # Check for zeros.     
        if W_m_paths[kdx,-1] <= tolerance and W_s_paths[kdx,-1] <= tolerance:
            both_zero_count_all = both_zero_count_all + 1.
            #W_m_final[jdx,kdx] = 0.
            #W_s_final[jdx,kdx] = 0.
        elif W_m_paths[kdx,-1] <= tolerance:
            W_m_zero_count_all = W_m_zero_count_all + 1.
            #W_m_final[jdx,kdx] = 0.
            #W_s_final[jdx,kdx] = W_s_paths[jdx,kdx,-1]
        elif W_s_paths[kdx,-1] <= tolerance:
            W_s_zero_count_all = W_s_zero_count_all + 1.
            #W_s_final[jdx,kdx] = 0.
            #W_m_final[jdx,kdx] = W_m_paths[jdx,kdx,-1]
        else: #neither is zero; save normally
            W_m_final_all[kdx, jdx] = W_m_paths[kdx,-1]
            W_s_final_all[kdx, jdx] = W_s_paths[kdx,-1]
        
        W_m_final_all[kdx, jdx] = W_m_paths[kdx,-1]
        W_s_final_all[kdx, jdx] = W_s_paths[kdx,-1]      
            
print('Zero water: ', 'W_m: ', W_m_zero_count_all, 'W_s: ', W_s_zero_count_all, 'both: ', both_zero_count_all)

### Individual read-ins for a given orbital distance.

In [ ]:
# Choose which orbital distance files will be read-in (Inner HZ, Mid HZ, Outer HZ).
# This should make it easier to make temporal plots. 
orb_idx = 0 #0-2
print(a_orb_labels[orb_idx])

In [ ]:
# Read in MO results (pre-cycling).

#tolerance = 1.29e16 #[kg]; if below this, set water = 0 --> DESICCATED

# Initial water content in each reservoir
M_MO_initial = np.zeros(len(M_init_array))
M_SM_initial = np.zeros(len(M_init_array))
M_atmo_initial = np.zeros(len(M_init_array))

# Final water content in each reservoir
M_MO_final = np.zeros(len(M_init_array))
M_SM_final = np.zeros(len(M_init_array))
M_atmo_final = np.zeros(len(M_init_array))

# Initial and final mantle temperatures, if needed.
T_MO_initial = np.zeros(len(M_init_array))
T_MO_final = np.zeros(len(M_init_array))
    
# Need length of data for the following arrays.
save_file = 'model2_MO_only_loss_M_init-2.0TO_' + str(a_orb_labels[orb_idx]) + '.txt'    
filename = os.path.join(save_path, save_file)
tmp_data = np.loadtxt(filename)

# Time array; same for all.
t_MO_array = tmp_data[:, 0]

# Overall evolution path for each reservoir & mantle temperature & surface temperature.
M_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
M_SM_paths = np.zeros((len(M_init_array),len(tmp_data)))
M_atmo_paths = np.zeros((len(M_init_array),len(tmp_data)))
r_sol_paths = np.zeros((len(M_init_array),len(tmp_data)))
T_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
TOA_flux_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))

loss_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
EL_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
DL_MO_paths = np.zeros((len(M_init_array),len(tmp_data)))
    
for kdx in range(0, len(M_init_array)):
                
    # Read in data files for each model, and save them as the evolutionary path. 
    save_file = 'model2_MO_only_loss_M_init-' + str(M_init_file[kdx]) + 'TO_' + str(a_orb_labels[orb_idx]) + '.txt'
        
    print(save_file)
    filename = os.path.join(save_path, save_file)
    tmp_data = np.loadtxt(filename)
    r_sol_paths[kdx,:] = tmp_data[:,1]
    T_MO_paths[kdx,:] = tmp_data[:,2]
    M_MO_paths[kdx,:] = tmp_data[:,3]
    M_SM_paths[kdx,:] = tmp_data[:,4]
    M_atmo_paths[kdx,:] = tmp_data[:,5]
    EL_MO_paths[kdx,:] = tmp_data[:,6]
    DL_MO_paths[kdx,:] = tmp_data[:,7]
    loss_MO_paths[kdx,:] = tmp_data[:,8]
    TOA_flux_MO_paths[kdx,:] = tmp_data[:,9]

    # Save initial and final values to corresponding arrays.
    M_MO_initial[kdx] = M_MO_paths[kdx,0]
    M_SM_initial[kdx] = M_SM_paths[kdx,0]
    M_atmo_initial[kdx] = M_atmo_paths[kdx,0]
    
    T_MO_initial[kdx] = T_MO_paths[kdx,0]
    T_MO_final[kdx] = T_MO_paths[kdx,-1]
    
    # Check for zeros.     
    #if W_m_paths[kdx,ldx,-1] <= tolerance and W_s_paths[kdx,ldx,-1] <= tolerance:
    #    both_zero_count_all = both_zero_count_all + 1.
        #W_m_final[jdx,kdx] = 0.
        #W_s_final[jdx,kdx] = 0.
    #elif W_m_paths[kdx,ldx,-1] <= tolerance:
    #    W_m_zero_count_all = W_m_zero_count_all + 1.
        #W_m_final[jdx,kdx] = 0.
        #W_s_final[jdx,kdx] = W_s_paths[jdx,kdx,-1]
    #elif W_s_paths[kdx,ldx,-1] <= tolerance:
    #    W_s_zero_count_all = W_s_zero_count_all + 1.
        #W_s_final[jdx,kdx] = 0.
        #W_m_final[jdx,kdx] = W_m_paths[jdx,kdx,-1]
    #else: #neither is zero; save normally
    #    W_m_final_all[jdx, kdx, ldx] = W_m_paths[kdx,ldx,-1]
    #    W_s_final_all[jdx, kdx, ldx] = W_s_paths[kdx,ldx,-1]
        
    M_MO_final[kdx] = M_MO_paths[kdx,-1]
    M_SM_final[kdx] = M_SM_paths[kdx,-1]     
    M_atmo_final[kdx] = M_atmo_paths[kdx,-1]
            
#print('Zero water: ', 'W_m: ', W_m_zero_count_all, 'W_s: ', W_s_zero_count_all, 'both: ', both_zero_count_all)

In [ ]:
#Read in cycling files (post-MO). 

tolerance = 1.29e16 #[kg]; if below this, set water = 0 --> DESICCATED

# Initial water content in each reservoir
W_m_initial = np.zeros(len(M_init_array))
W_s_initial = np.zeros(len(M_init_array))

# Final water content in each reservoir
W_m_final = np.zeros(len(M_init_array))
W_s_final = np.zeros(len(M_init_array))

# Initial and final mantle temperatures, if needed.
T_initial = np.zeros(len(M_init_array))
T_final = np.zeros(len(M_init_array))

# How many reach exactly zero in one/both reservoir/s at the end?
W_m_zero_count = 0.
W_s_zero_count = 0.
both_zero_count = 0.
    
# Need length of data for the following arrays.
save_file = 'model2_combined_MO_cycling_loss_M_init-2.0TO_' + str(a_orb_labels[orb_idx]) + '.txt'   
filename = os.path.join(save_path, save_file)
tmp_data = np.loadtxt(filename)

# Time array; same for all.
t_array = tmp_data[:, 0]

# Overall evolution path for each reservoir & mantle temperature & surface temperature.
W_m_paths = np.zeros((len(M_init_array),len(tmp_data)))
W_s_paths = np.zeros((len(M_init_array),len(tmp_data)))
T_paths = np.zeros((len(M_init_array),len(tmp_data)))
T_surf_paths = np.zeros((len(M_init_array),len(tmp_data)))
TOA_flux_paths = np.zeros((len(M_init_array),len(tmp_data)))

# Overall evolution path for degassing and regassing rates.
degas_paths = np.zeros((len(M_init_array),len(tmp_data)))
regas_paths = np.zeros((len(M_init_array),len(tmp_data)))
loss_paths = np.zeros((len(M_init_array),len(tmp_data)))
EL_paths = np.zeros((len(M_init_array),len(tmp_data)))
DL_paths = np.zeros((len(M_init_array),len(tmp_data)))
    
for kdx in range(0, len(M_init_array)):
                
    # Read in data files for each model, and save them as the evolutionary path. 
    save_file = 'model2_combined_MO_cycling_loss_M_init-' + str(M_init_file[kdx]) + 'TO_' + str(a_orb_labels[orb_idx]) + '.txt'
            
    print(save_file)
    filename = os.path.join(save_path, save_file)
    tmp_data = np.loadtxt(filename)
    T_paths[kdx,:] = tmp_data[:,1]
    W_m_paths[kdx,:] = tmp_data[:,2]
    W_s_paths[kdx,:] = tmp_data[:,3]
    degas_paths[kdx,:] = tmp_data[:,4]
    regas_paths[kdx,:] = tmp_data[:,5]
    T_surf_paths[kdx,:] = tmp_data[:,6]
    TOA_flux_paths[kdx,:] = tmp_data[:,8]
    loss_paths[kdx,:] = tmp_data[:,9]
    EL_paths[kdx,:] = tmp_data[:,10]
    DL_paths[kdx,:] = tmp_data[:,11]
    #DL_paths[jdx,kdx,:] = tmp_data[:,11]

    # Save initial and final values to corresponding arrays.
    W_m_initial[kdx] = W_m_paths[kdx,0]
    W_s_initial[kdx] = W_s_paths[kdx,0]
    
    T_initial[kdx] = T_paths[kdx,0]
    T_final[kdx] = T_paths[kdx,-1]
    
    # Check for zeros.     
    if W_m_paths[kdx,-1] <= tolerance and W_s_paths[kdx,-1] <= tolerance:
        both_zero_count = both_zero_count + 1.
        #W_m_final[jdx,kdx] = 0.
        #W_s_final[jdx,kdx] = 0.
    elif W_m_paths[kdx,-1] <= tolerance:
        W_m_zero_count = W_m_zero_count + 1.
        #W_m_final[jdx,kdx] = 0.
        #W_s_final[jdx,kdx] = W_s_paths[jdx,kdx,-1]
    elif W_s_paths[kdx,-1] <= tolerance:
        W_s_zero_count = W_s_zero_count + 1.
        #W_s_final[jdx,kdx] = 0.
        #W_m_final[jdx,kdx] = W_m_paths[jdx,kdx,-1]
    else: #neither is zero; save normally
        W_m_final[kdx] = W_m_paths[kdx,-1]
        W_s_final[kdx] = W_s_paths[kdx,-1]
        
    W_m_final[kdx] = W_m_paths[kdx,-1]
    W_s_final[kdx] = W_s_paths[kdx,-1]      
            
print('Zero water: ', 'W_m: ', W_m_zero_count, 'W_s: ', W_s_zero_count, 'both: ', both_zero_count)

# Plot all results.

# Make a temporal plot (for an individual result for now):
# MO, linear in time, scaled to solidification timescale, on left.
# Cycling, log time, on right.

In [ ]:
init_idx = 2 #0-3; initial surface water: {2,4,6,8}

#t_MO_array already defined
T_MO_array = T_MO_paths[init_idx,:]
M_MO_array = M_MO_paths[init_idx,:]
M_SM_array = M_SM_paths[init_idx,:]
M_atmo_array = M_atmo_paths[init_idx,:]
r_array = r_sol_paths[init_idx,:]
TOA_flux_MO_array = TOA_flux_MO_paths[init_idx,:]
EL_MO_array = EL_MO_paths[init_idx,:]
DL_MO_array = DL_MO_paths[init_idx,:]
loss_MO_array = loss_MO_paths[init_idx,:]
T_surf_MO_array = np.ones(len(TOA_flux_MO_array))*1800. #fixed during RG

#t_array already defined
T_array = T_paths[init_idx,:]
W_m_array = W_m_paths[init_idx,:]
W_s_array = W_s_paths[init_idx,:]
degas_array = degas_paths[init_idx,:]
regas_array = regas_paths[init_idx,:]
T_surf_array = T_surf_paths[init_idx,:]
loss_array = loss_paths[init_idx,:]
TOA_flux_array = TOA_flux_paths[init_idx,:]
EL_array = EL_paths[init_idx,:]
DL_array = DL_paths[init_idx,:]

print('M_init =', M_init_array[init_idx]/1.4e21, 'TO, a_orb =', a_orb_labels[orb_idx], ' = ', a_orb_array[orb_idx], 'AU')
print('W_s_final =', W_s_final[init_idx]/1.4e21, 'TO')
#plt.plot(t_array[0:10], W_s_array[0:10]/1.4e21, color='b')

In [ ]:
des_idx = np.argmin(W_m_array<1.29e16)
print(W_m_array[0])
#des = np.where(W_m_array<1.29e16)[0][0]
#print(type(des))
#des_idx = des[0][0]
#print(des)
print(des_idx)

In [ ]:
from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3])

ax1 = plt.subplot(gs[0])
# Plot time data in reverse, then flip x-axis and change the axis labels.
ax1 = plt.subplot(gs[0])
plt.plot(t_MO_array[::-1]/year/1.0e6, (M_atmo_array+M_MO_array+M_SM_array)/1.4e21, linewidth=4, linestyle='-.', color='k')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere')
plt.plot(t_MO_array[::-1]/year/1.0e6, (M_MO_array)/1.4e21, linewidth=3, color='r', linestyle='--', label='Magma Ocean')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
ax1.set_xscale('log')
plt.axvspan(t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6, alpha=0.2, color='grey')
plt.xlim([t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6])
plt.ylim([-0.2, 6.2]) #nominal, 6 TO case
plt.gca().invert_xaxis()
plt.xlabel('Time [Myr]', fontsize=30)
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=30)
#ax1.set_xticks([0.01, 1., 100.]) #comment this in/out if there is an issue
ax1.set_xticks([1.]) #comment this in/out if there is an issue
plt.gca().set_xticklabels((round(tau_RG_array[orb_idx]/year/1.0e6,2))-plt.gca().get_xticks())

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
## Re-colour/change linestyle if/when SURFACE becomes desiccated during cycling.
#des_surf_idx = np.where(W_s_array<1.29e16)[0][0]
#plt.semilogx(t_array[0:des_surf_idx-1]/year/1.0e9, W_s_array[0:des_surf_idx-1]/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere/Surface')
#plt.semilogx(t_array[des_surf_idx:]/year/1.0e9, W_s_array[des_surf_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.') #, label='Desiccated')
## If surface doesn't become desiccated, just plot as-is.
#plt.semilogx(t_array/year/1.0e9, (W_m_array+W_s_array)/1.4e21, linewidth=4, linestyle='-.', color='k')
plt.plot(10., 10., linewidth=4, linestyle=(0, (6, 1)), color='k', label='Total Water')
#plt.semilogx(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere/Surface')
plt.plot(10., 10., linewidth=3, color='b', linestyle='-', label='Atmosphere/Surface')
plt.plot(10., 10., linewidth=3, color='r', linestyle='--', label='Magma Ocean')
## Re-colour/change linestyle if/when MANTLE becomes desiccated during cycling.
#des_idx = np.where(W_m_array<1.29e16)[0][0]
#plt.semilogx(t_array[0:des_idx-1]/year/1.0e9, W_m_array[0:des_idx-1]/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
#plt.semilogx(t_array[des_idx:]/year/1.0e9, W_m_array[des_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.', label='Desiccated Mantle')
## If mantle doesn't become desiccated, just plot as-is.
#plt.semilogx(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
W_m_plot_array = np.ones(len(W_m_array))*W_m_array[0]
plt.semilogx(t_array/year/1.0e9, W_m_plot_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')

#plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([t_MO_array[-1]/year/1.0e9, 5.0])
plt.ylim([-0.2, 6.2]) #nominal, 6 TO case
plt.xticks([1,2,3,4,5], [1,2,3,4,5])
#plt.ylim([-0.2, 4.2])
plt.xlabel('Time [Gyr]', fontsize=30)
#plt.legend(loc='center left', fontsize=24, framealpha=1)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('Temporal_MO_Cycling_Loss_M8_6TO_RunawayGH_C0d001.pdf', bbox_inches='tight')

### Alternate way to plot the above: normal log-scale or linear MO panel.

In [ ]:
from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3])

ax1 = plt.subplot(gs[0])
plt.plot(t_MO_array/year/1.0e6, (M_atmo_array+M_MO_array+M_SM_array)/1.4e21, linewidth=4, linestyle='-.', color='k')
plt.plot(t_MO_array/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere')
plt.plot(t_MO_array/year/1.0e6, (M_MO_array)/1.4e21, linewidth=3, color='r', linestyle='--', label='Magma Ocean')
plt.plot(t_MO_array/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
ax1.set_xscale('log')
plt.axvspan(t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6, alpha=0.2, color='grey')
plt.xlim([t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6])
plt.ylim([-0.2, 6.2]) #nominal, 6 TO case
#plt.gca().invert_xaxis()
plt.xlabel('Time [Myr]', fontsize=24)
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=24)
#ax1.set_xticks([0.01, 1., 100.]) #comment this in/out if there is an issue
#plt.gca().set_xticklabels((round(tau_RG_array[orb_idx]/year/1.0e6,2))-plt.gca().get_xticks())

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
## Re-colour/change linestyle if/when SURFACE becomes desiccated during cycling.
#des_surf_idx = np.where(W_s_array<1.29e16)[0][0]
#plt.semilogx(t_array[0:des_surf_idx-1]/year/1.0e9, W_s_array[0:des_surf_idx-1]/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere/Surface')
#plt.semilogx(t_array[des_surf_idx:]/year/1.0e9, W_s_array[des_surf_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.') #, label='Desiccated')
## If surface doesn't become desiccated, just plot as-is.
#plt.semilogx(t_array/year/1.0e9, (W_m_array+W_s_array)/1.4e21, linewidth=4, linestyle='-.', color='k')
plt.plot(10., 10., linewidth=4, linestyle=(0, (6, 1)), color='k', label='Total Water')
#plt.semilogx(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere/Surface')
plt.plot(10., 10., linewidth=3, color='b', linestyle='-', label='Atmosphere/Surface')
plt.plot(10., 10., linewidth=3, color='r', linestyle='--', label='Magma Ocean')
## Re-colour/change linestyle if/when MANTLE becomes desiccated during cycling.
#des_idx = np.where(W_m_array<1.29e16)[0][0]
#plt.semilogx(t_array[0:des_idx-1]/year/1.0e9, W_m_array[0:des_idx-1]/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
#plt.semilogx(t_array[des_idx:]/year/1.0e9, W_m_array[des_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.', label='Desiccated Mantle')
## If mantle doesn't become desiccated, just plot as-is.
#plt.semilogx(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
W_m_plot_array = np.ones(len(W_m_array))*W_m_array[0]
plt.semilogx(t_array/year/1.0e9, W_m_plot_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')

#plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([t_MO_array[-1]/year/1.0e9, 5.0])
plt.ylim([-0.2, 6.2]) #nominal, 6 TO case
#plt.ylim([-0.2, 4.2])
plt.xlabel('Time [Gyr]', fontsize=24)
#plt.legend(loc='center left', fontsize=24, framealpha=1)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('TEST_Temporal_MO_Cycling_Loss_M8_6TO_RunawayGH_C0d001_normallogMO.pdf', bbox_inches='tight')

In [ ]:
print('End of magma ocean: ', M_atmo_array[-1]/1.4e21)
print('Start of cycling:   ', W_s_array[0]/1.4e21)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3])

ax1 = plt.subplot(gs[0])
# Plot time data in reverse, then flip x-axis and change the axis labels.
ax1 = plt.subplot(gs[0])
plt.plot(t_MO_array[::-1]/year/1.0e6, (M_atmo_array+M_MO_array+M_SM_array)/1.4e21, linewidth=4, linestyle='-.', color='k')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere')
plt.plot(t_MO_array[::-1]/year/1.0e6, (M_MO_array)/1.4e21, linewidth=3, color='r', linestyle='--', label='Magma Ocean')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
ax1.set_xscale('log')
plt.axvspan(t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6, alpha=0.2, color='grey')
plt.xlim([t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6])
plt.ylim([-0.2, 6.2]) #nominal, 6 TO case
plt.gca().invert_xaxis()
plt.xlabel('Time [Myr]', fontsize=30)
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=30)
#ax1.set_xticks([0.01, 1., 100.]) #comment this in/out if there is an issue
ax1.set_xticks([1.]) #comment this in/out if there is an issue
plt.gca().set_xticklabels((round(tau_RG_array[orb_idx]/year/1.0e6,2))-plt.gca().get_xticks())

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
## Re-colour/change linestyle if/when SURFACE becomes desiccated during cycling.
#des_surf_idx = np.where(W_s_array<1.29e16)[0][0]
#plt.semilogx(t_array[0:des_surf_idx-1]/year/1.0e9, W_s_array[0:des_surf_idx-1]/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere/Surface')
#plt.semilogx(t_array[des_surf_idx:]/year/1.0e9, W_s_array[des_surf_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.') #, label='Desiccated')
## If surface doesn't become desiccated, just plot as-is.
#plt.semilogx(t_array/year/1.0e9, (W_m_array+W_s_array)/1.4e21, linewidth=4, linestyle='-.', color='k')
plt.plot(10., 10., linewidth=4, linestyle=(0, (6, 1)), color='k', label='Total Water')
#plt.semilogx(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere/Surface')
plt.plot(10., 10., linewidth=3, color='b', linestyle='-', label='Atmosphere/Surface')
plt.plot(10., 10., linewidth=3, color='r', linestyle='--', label='Magma Ocean')
## Re-colour/change linestyle if/when MANTLE becomes desiccated during cycling.
#des_idx = np.where(W_m_array<1.29e16)[0][0]
#plt.semilogx(t_array[0:des_idx-1]/year/1.0e9, W_m_array[0:des_idx-1]/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
#plt.semilogx(t_array[des_idx:]/year/1.0e9, W_m_array[des_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.', label='Desiccated Mantle')
## If mantle doesn't become desiccated, just plot as-is.
plt.semilogx(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')

##W_m_plot_array = np.ones(len(W_m_array))*W_m_array[0]
##plt.semilogx(t_array/year/1.0e9, W_m_plot_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')

#plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([t_MO_array[-1]/year/1.0e9, 5.0])
plt.ylim([-0.2, 6.2]) #nominal, 6 TO case
plt.xticks([1,2,3,4,5], [1,2,3,4,5])
#plt.ylim([-0.2, 4.2])
plt.xlabel('Time [Gyr]', fontsize=30)
#plt.legend(loc='center left', fontsize=24, framealpha=1)


axins = inset_axes(ax1,
                   width=2.5,  
                   height=5,
                   loc=3,
                   bbox_to_anchor=(0.17, 0.15, 0.5, 0.5),
                   bbox_transform=ax1.transAxes,
                   borderpad=2,
                   )
plt.semilogx(t_MO_array/year/1.0e6, (M_atmo_array+M_MO_array+M_SM_array)/1.4e21, linewidth=4, linestyle='-.', color='k')
plt.semilogx(t_MO_array/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere')
plt.semilogx(t_MO_array/year/1.0e6, (M_MO_array)/1.4e21, linewidth=3, color='r', linestyle='--', label='Magma Ocean')
plt.semilogx(t_MO_array/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
plt.axvspan(t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6, alpha=0.2, color='grey')
plt.xlim([t_MO_array[0]/year/1.0e6, t_MO_array[-1]/year/1.0e6])
plt.ylim([-0.2, 6.2]) #nominal, 6 TO case
plt.xticks([0.1, 1, 10], [0.1, 1, 10], rotation=90)
#plt.gca().invert_xaxis()
#plt.xlabel('Time [Myr]', fontsize=24)
#plt.ylabel('Water Inventory [Earth Oceans]', fontsize=24)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('Temporal_MO_Cycling_Loss_M8_6TO_RunawayGH_C0d001.pdf', bbox_inches='tight')

## Evolution of various fluxes and rates over time.

In [ ]:
# Calculate F_XUV and F_XUV_MO, to be plotted in the following panel.
F_XUV_MO_array = np.zeros(len(t_MO_array))
F_XUV_array = np.zeros(len(t_array))

for idx in range(0,len(F_XUV_MO_array)):
    
    F_XUV_MO_array[idx] = F_XUV(t_MO_array[idx], M, a_orb, params1, params2, params3)

for idx in range(0,len(F_XUV_array)):
    
    F_XUV_array[idx] = F_XUV(t_array[idx], M, a_orb, params1, params2, params3)
    
# Saturation timescale, to be plotted as vertical line
t_sat = 1.0e9*year

In [ ]:
# Plot TOA flux over time, and a horizontal line with Goldblatt/Watson limit of 300 W/m^2.
# First, find where runaway greenhouse ends, to be plotted as a vertical line.

%matplotlib inline
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(14,10))
plt.tight_layout()
ax = [fig.add_subplot(2,2,i+1) for i in range(4)]
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(2, 2, 1)
plt.loglog(t_MO_array/year/1.0e9, TOA_flux_MO_array/4, linewidth=3, color='b')
plt.loglog(t_array/year/1.0e9, TOA_flux_array/4, linewidth=3, color='b') #divide by 4 for sphere
plt.axhline(325./(1.-alb), color='k', linestyle='--', label='Simpson-Nakajima limit')
plt.text(0.00002, (325./(1.-alb))+30, 'Simpson-Nakajima limit', color='k', fontsize=15)
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.xticks([],[])
plt.ylabel(r'TOA Flux/4 [W/m$^2$]', fontsize=16)
#plt.legend(fontsize=14, loc='upper right')
plt.axvspan(1.0e-5, tau_RG_array[orb_idx]/year/1.0e9, alpha=0.2, color='grey')

ax2 = plt.subplot(2, 2, 2)
plt.loglog(t_MO_array/year/1.0e9, F_XUV_MO_array, linewidth=3, color='k')
plt.loglog(t_array/year/1.0e9, F_XUV_array, linewidth=3, color='k')
plt.axvline(t_sat/year/1.0e9, linestyle='--', linewidth=2, color='k')
plt.text(0.75, 0.5, r'$t_{{\mathrm{{sat}}}} = 1$ Gyr', color='k', rotation=90, rotation_mode='anchor', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.xticks([],[])
plt.ylabel(r'$F_{{\mathrm{{XUV}}}}$ [W/m$^2$]', fontsize=16, labelpad=2.0)
plt.axvspan(1.0e-5, tau_RG_array[orb_idx]/year/1.0e9, alpha=0.2, color='grey')

ax3 = plt.subplot(2, 2, 3)
plt.loglog(t_MO_array/year/1.0e9, T_surf_MO_array, linewidth=3, color='k')
plt.loglog(t_array/year/1.0e9, T_surf_array, linewidth=3, color='k')
plt.loglog([t_MO_array[-1]/year/1.0e9, t_array[0]/year/1.0e9], [T_surf_MO_array[-1], T_surf_array[0]], linewidth=3, color='k')
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.ylabel(r'$T_{{\mathrm{{surf}}}}$ [K]', fontsize=16)
#print('Minimum surface temperature = ', np.min(T_surf_array), 'K')
plt.axvspan(1.0e-5, tau_RG_array[orb_idx]/year/1.0e9, alpha=0.2, color='grey')

ax4 = plt.subplot(2, 2, 4)
plt.loglog(t_MO_array/year/1.0e9, EL_MO_array, linewidth=3, color='r', linestyle='-.')
plt.loglog(t_MO_array/year/1.0e9, DL_MO_array, linewidth=3, color='g', linestyle='--')
plt.loglog(t_array/year/1.0e9, EL_array, linewidth=3, color='r', linestyle='-.', label='energy-limited')
plt.loglog(t_array/year/1.0e9, DL_array, linewidth=3, color='g', linestyle='--', label='diffusion-limited')
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#Plot in bold the actual loss prescription being followed, based on TOA flux runaway greenhouse limit.
plt.loglog(t_MO_array/year/1.0e9, loss_MO_array, linewidth=3, color='k')
plt.loglog(t_array/year/1.0e9, loss_array, linewidth=3, color='k', label='loss to space')
plt.loglog([t_MO_array[-1]/year/1.0e9, t_array[0]/year/1.0e9], [loss_MO_array[-1], loss_array[0]], linewidth=3, color='k')
plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.ylabel('Escape rate [kg/s]', fontsize=16)
plt.legend(fontsize=14, loc='center left')
plt.axvspan(1.0e-5, tau_RG_array[orb_idx]/year/1.0e9, alpha=0.2, color='grey')

#plt.savefig('Temporal_TOAFlux_LossRates_6TO_OuterHZ_loglog.pdf', bbox_inches='tight')

In [ ]:
# Same as above, but include simulations for ALL 3 HZ locations in right panel.
from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3])

ax1 = plt.subplot(gs[0])
# Plot time data in reverse, then flip x-axis and change the axis labels.
plt.plot(t_MO_array[::-1]/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='-', label='Atmosphere')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_MO_array/1.4e21, linewidth=3, color='r', linestyle='--', label='Magma Ocean')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='chocolate', linestyle=':', label='Solid Mantle')
# Include vertical lines indicating end of RG, in same linestyle as HZ location below.
plt.axvline(x=0.022*1.0e3, linewidth=2, linestyle='-.', color='k') #Outer HZ
ax1.set_xscale('log')
plt.ylim([-0.2, 6.2])
plt.xlabel('Time [Myr]', fontsize=16)
plt.xlim([2e-3,tau_MO_file[tau_idx]])
plt.gca().invert_xaxis()
plt.gca().set_xticklabels(tau_MO_file[tau_idx]-plt.gca().get_xticks())
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=16)
#plt.legend(loc='center left', fontsize=16)

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
## Re-colour/change linestyle if/when SURFACE becomes desiccated during cycling.
#des_surf_idx = np.where(W_s_array<1.29e16)[0][0]
#plt.semilogx(t_array[0:des_surf_idx-1]/year/1.0e9, W_s_array[0:des_surf_idx-1]/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere/Surface')
#plt.semilogx(t_array[des_surf_idx:]/year/1.0e9, W_s_array[des_surf_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.') #, label='Desiccated')
## If surface doesn't become desiccated, just plot as-is.

for plt_idx in range(3): #orb_idx = [0,1,2] for Inner/Mid/Outer HZ, respectively
    
    if plt_idx == 0: #inner HZ
        ls = '--'
        exit_time = 0.195 #Gyr
    elif plt_idx == 1: #mid HZ
        ls = ':'
        exit_time = 0.056 #Gyr
    else:
        ls = '-.'
        exit_time = 0.022 #Gyr

    plt.semilogx(t_array/year/1.0e9, W_s_paths[init_idx,plt_idx,:]/1.4e21, linewidth=3, color='b', linestyle=ls)
    
    #plt.plot(10., 10., linewidth=3, color='orange', linestyle='-')
    ## Re-colour/change linestyle if/when MANTLE becomes desiccated during cycling.
    #W_m_array = W_m_paths[init_idx,plt_idx,:]
    #des_idx = np.where(W_m_array<1.29e16)[0][0]
    #if des_idx > 0.:
    #    plt.semilogx(t_array[0:des_idx-1]/year/1.0e9, W_m_paths[init_idx,plt_idx,0:des_idx-1]/1.4e21, linewidth=3, color='r', linestyle=':', label='Solid Mantle')
    #    plt.semilogx(t_array[des_idx:]/year/1.0e9, W_m_paths[init_idx,plt_idx,des_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.', label='Desiccated')
    ## If mantle doesn't become desiccated, just plot as-is.
    #else:
    plt.semilogx(t_array/year/1.0e9, W_m_paths[init_idx,plt_idx,:]/1.4e21, linewidth=3, color='chocolate', linestyle=ls)
    
# Include vertical lines indicating end of RG, in same linestyle as HZ location below.
plt.axvline(x=0.195, linewidth=2, linestyle='--', color='k') #Inner HZ 
plt.axvline(x=0.056, linewidth=2, linestyle=':', color='k') #Mid HZ

# Labels for HZ distances
plt.plot(10., 10., linewidth=3, color='k', linestyle='--', label='Inner HZ')
plt.plot(10., 10., linewidth=3, color='k', linestyle=':', label='Mid HZ')
plt.plot(10., 10., linewidth=3, color='k', linestyle='-.', label='Outer HZ')

#plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([tau_MO_file[tau_idx]/1000., 5.0])
plt.ylim([-0.2, 6.2])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.legend(loc='center left', fontsize=16)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('Temporal_MO_Cycling_Loss_M5_6TO_MaxGH_100Myr.pdf', bbox_inches='tight')

In [ ]:
from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]

# XXXX DIFFICULT TO DO -- FIGURE OUT LATER??? XXXX
# Set width ratios, based on solidification timescales: 10, 50, 100.
#if tau_idx == 0: #10 Myr; smallest left panel
#    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 5]) 
#elif tau_idx == 1: #50 Myr
#    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 4]) 
#elif tau_idx == 2: #100 Myr; largest left panel
#    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3]) 
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3])

ax1 = plt.subplot(gs[0])
# Plot time data in reverse, then flip x-axis and change the axis labels.
plt.semilogy(t_MO_array[::-1]/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere')
plt.semilogy(t_MO_array[::-1]/year/1.0e6, M_MO_array/1.4e21, linewidth=3, color='orange', linestyle='-', label='Magma Ocean')
plt.semilogy(t_MO_array[::-1]/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Solid Mantle')
ax1.set_xscale('log')
plt.ylim([1.0e-6, 10.])
plt.xlabel('Time [Myr]', fontsize=16)
plt.xlim([2e-3,tau_MO_file[tau_idx]])
plt.gca().invert_xaxis()
plt.gca().set_xticklabels(tau_MO_file[tau_idx]-plt.gca().get_xticks())
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=16)
plt.legend(loc='lower right', fontsize=16)

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
plt.loglog(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.loglog(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([tau_MO_file[tau_idx]/1000., 5.0])
plt.ylim([1.0e-6, 10.])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.legend(loc='center', fontsize=16)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('Temporal_MO_Cycling_Loss_M5_6TO_MidHZ_50Myr_loglog.pdf', bbox_inches='tight')


In [ ]:
print(M_atmo_array[-1]/1.4e21)
print(W_s_array[0]/1.4e21)

In [ ]:
# Same as above, but water inventory is logged.

from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 4]) 

ax1 = plt.subplot(gs[0])
plt.semilogy(t_MO_array/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere')
plt.semilogy(t_MO_array/year/1.0e6, M_MO_array/1.4e21, linewidth=3, color='orange', linestyle='-', label='Magma Ocean')
plt.semilogy(t_MO_array/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Solid Mantle')
plt.xlim([0.,tau_MO_file[tau_idx]])
plt.ylim([1.0e-6, 10.])
plt.xlabel('Time [Myr]', fontsize=16)
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=16)
plt.legend(loc='lower center', fontsize=16)

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
plt.loglog(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.loglog(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([tau_MO_file[tau_idx]/1000., 5.0])
plt.ylim([1.0e-6, 10.])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.legend(loc='center', fontsize=16)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('Temporal_MO_Cycling_Loss_M5_6TO_MidHZ_50Myr_linear.pdf', bbox_inches='tight')


## The cells below show how to make temporal plots, specifically for 50 Myr. These have been updated above to be automatic after read-in.

In [ ]:
from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3]) 

ax1 = plt.subplot(gs[0])
# Plot time data in reverse, then flip x-axis and change the axis labels.
plt.plot(t_MO_array[::-1]/year/1.0e6, M_MO_array/1.4e21, linewidth=3, color='orange', linestyle='-', label='Magma Ocean')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Solid Mantle')
plt.plot(t_MO_array[::-1]/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere')
ax1.set_xscale('log')
plt.ylim([-0.2, 6.2])
plt.xlabel('Time [Myr]', fontsize=16)
plt.xlim([2e-3,50.])
plt.gca().invert_xaxis()
plt.gca().set_xticklabels(50.-plt.gca().get_xticks())
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=16)
plt.legend(loc='center left', fontsize=16)

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
plt.semilogx(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
plt.semilogx(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([50.0e6/1.0e9, 5.0])
plt.ylim([-0.2, 6.2])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.legend(loc='center left', fontsize=16)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('Temporal_MO_Cycling_Loss_M5_6TO_50Myr.pdf', bbox_inches='tight')


In [ ]:
# Same as above, but water inventory is logged.

from matplotlib import gridspec
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(16,8))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3]) 

ax1 = plt.subplot(gs[0])
plt.semilogy(t_MO_array/year/1.0e6, M_MO_array/1.4e21, linewidth=3, color='orange', linestyle='-', label='Magma Ocean')
plt.semilogy(t_MO_array/year/1.0e6, M_SM_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Solid Mantle')
plt.semilogy(t_MO_array/year/1.0e6, M_atmo_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Atmosphere')
plt.xlim([0.,50.])
plt.ylim([1.0e-6, 10.])
plt.xlabel('Time [Myr]', fontsize=16)
plt.ylabel('Water Inventory [Earth Oceans]', fontsize=16)
plt.legend(loc='lower center', fontsize=16)

ax2 = plt.subplot(gs[1])
ax2.yaxis.tick_right()
plt.loglog(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
plt.loglog(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')
plt.xlim([50.0e6/1.0e9, 5.0])
plt.ylim([1.0e-6, 10.])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.legend(loc='center left', fontsize=16)

plt.tight_layout()
fig.subplots_adjust(wspace=0)
#plt.savefig('Temporal_MO_Cycling_Loss_M5_6TO_50Myr_linear.pdf', bbox_inches='tight')

# Plot the simulation results, as a function of time.
## Look at some of the paths individually.

In [ ]:
init_idx = 2 #0-3; initial surface water: {2,4,6,8}
orb_idx = 1 #0-2; orbital distance in HZ: {RG, Mid, MG}

T_array = T_paths[init_idx,orb_idx,:]
W_m_array = W_m_paths[init_idx,orb_idx,:]
W_s_array = W_s_paths[init_idx,orb_idx,:]
degas_array = degas_paths[init_idx,orb_idx,:]
regas_array = regas_paths[init_idx,orb_idx,:]
T_surf_array = T_surf_paths[init_idx,orb_idx,:]
loss_array = loss_paths[init_idx,orb_idx,:]
TOA_flux_array = TOA_flux_paths[init_idx,orb_idx,:]
EL_array = EL_paths[init_idx,orb_idx,:]
DL_array = DL_paths[init_idx,orb_idx,:]

# Need ones from MO files as well for flux 4-panel plot.
loss_MO_array = loss_MO_paths[init_idx,orb_idx,:]
TOA_flux_MO_array = TOA_flux_MO_paths[init_idx,orb_idx,:]
EL_MO_array = EL_MO_paths[init_idx,orb_idx,:]
DL_MO_array = DL_MO_paths[init_idx,orb_idx,:]
T_surf_MO_array = np.ones(len(loss_MO_array))*1800.

print('tau_MO =', tau_MO_file[tau_idx], 'Myr, M_init =', M_init_array[init_idx]/1.4e21, 'TO, a_orb =', a_orb_labels[orb_idx])
print('W_s_final =', W_s_final[init_idx, orb_idx]/1.4e21, 'TO')
#plt.plot(t_array[0:10], W_s_array[0:10]/1.4e21, color='b')

In [ ]:
%matplotlib inline
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(8.5,10))
plt.tight_layout()
ax = [fig.add_subplot(2,1,i+1) for i in range(2)]
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(2,1,1)
#plt.loglog(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
#plt.loglog(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.semilogx(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
# Re-colour/change linestyle when mantle becomes desiccated during cycling.
des_idx = np.where(W_m_array<1.29e16)[0][0]
plt.semilogx(t_array[0:des_idx-1]/year/1.0e9, W_m_array[0:des_idx-1]/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
plt.semilogx(t_array[des_idx:]/year/1.0e9, W_m_array[des_idx:]/1.4e21, linewidth=3, color='k', linestyle='-.', label='Desiccated')
#plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')

plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'Water Mass [Earth Oceans]', fontsize=16)
plt.legend(loc='center left', fontsize=16)

ax2 = plt.subplot(2,1,2)
plt.loglog(t_array/year/1.0e9, loss_array/1.4e21*year*1.0e9, linewidth=3, color='k', linestyle='-.', label='Loss to Space')
plt.loglog(t_array/year/1.0e9, regas_array/1.4e21*year*1.0e9, linewidth=3, color='r', linestyle=':', label='Regassing')
plt.loglog(t_array/year/1.0e9, degas_array/1.4e21*year*1.0e9, linewidth=3, color='b', linestyle='--', label='Degassing')

plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'Cycling Rate [Earth Oceans/Gyr]', fontsize=16)
plt.legend(loc='lower right', fontsize=16)
#plt.ylim([-0.05,650.])
#plt.legend(bbox_to_anchor=(0.0, 1.0), ncol=2, fontsize=16)

#plt.savefig('PostMO_Cycling_Loss_D0d2_6TO_MidHZ_50Myr_logrates.pdf', bbox_inches='tight')


In [ ]:
# Same plot but log-log

%matplotlib inline
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(8.5,10))
plt.tight_layout()
ax = [fig.add_subplot(2,1,i+1) for i in range(2)]
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(2,1,1)
plt.loglog(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.loglog(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
#plt.semilogx(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
#plt.semilogx(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3, label='Desiccation Limit')

plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'Water Mass [TO]', fontsize=16)
plt.legend(loc='center right', fontsize=16)

ax2 = plt.subplot(2,1,2)
plt.loglog(t_array/year/1.0e9, loss_array/1.4e21*year*1.0e9, linewidth=3, color='k', linestyle='-.', label='Loss to Space')
plt.loglog(t_array/year/1.0e9, regas_array/1.4e21*year*1.0e9, linewidth=3, color='r', linestyle=':', label='Regassing')
plt.loglog(t_array/year/1.0e9, degas_array/1.4e21*year*1.0e9, linewidth=3, color='b', linestyle='--', label='Degassing')

plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'Cycling Rate [TO/Gyr]', fontsize=16)
plt.legend(loc='lower right', fontsize=16)
#plt.ylim([-0.05,650.])
#plt.legend(bbox_to_anchor=(0.0, 1.0), ncol=2, fontsize=16)

#plt.savefig('PostMO_Cycling_Loss_6TO_MidHZ_50Myr_loglog.pdf', bbox_inches='tight')

In [ ]:
# Calculate F_XUV and F_XUV_MO, to be plotted in the following panel.
F_XUV_MO_array = np.zeros(len(t_MO_array))
F_XUV_array = np.zeros(len(t_array))

for idx in range(0,len(F_XUV_MO_array)):
    
    F_XUV_MO_array[idx] = F_XUV(t_MO_array[idx], M, a_orb, params1, params2, params3)

for idx in range(0,len(F_XUV_array)):
    
    F_XUV_array[idx] = F_XUV(t_array[idx], M, a_orb, params1, params2, params3)
    
# Saturation timescale, to be plotted as vertical line
t_sat = 1.0e9*year

In [ ]:
# Plot TOA flux over time, and a horizontal line with Goldblatt/Watson limit of 300 W/m^2.
# First, find where runaway greenhouse ends, to be plotted as a vertical line.
for idx in range(0, len(t_array)):
    
    if TOA_flux_array[idx]/4. < 325./(1.-alb):
        
        exit_time = t_array[idx]
        exit_idx = idx
        break

%matplotlib inline
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(14,10))
plt.tight_layout()
ax = [fig.add_subplot(2,2,i+1) for i in range(4)]
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(2, 2, 1)
plt.loglog(t_array/year/1.0e9, TOA_flux_array/4, linewidth=3, color='b') #divide by 4 for sphere
#plt.axhline(325./(1.-alb), color='k', linestyle='--', label='Simpson-Nakajima limit')
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'TOA Flux/4 [W/m$^2$]', fontsize=16)
#plt.legend(fontsize=16, loc='center right')
plt.axvspan(t_array[0]/year/1.0e9, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

ax2 = plt.subplot(2, 2, 2)
plt.loglog(t_array/year/1.0e9, F_XUV_array, linewidth=3, color='k')
plt.axvline(t_sat/year/1.0e9, linestyle='--', linewidth=2, color='k')
plt.text(0.9, 0.07, r'$t_{{\mathrm{{sat}}}} = 1$ Gyr', color='k', rotation=90, rotation_mode='anchor', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'$F_{{\mathrm{{XUV}}}}$ [W/m$^2$]', fontsize=16, labelpad=2.0)
plt.axvspan(t_array[0]/year/1.0e9, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

ax3 = plt.subplot(2, 2, 3)
plt.loglog(t_array/year/1.0e9, T_surf_array, linewidth=3)
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.ylabel(r'$T_{{\mathrm{{surf}}}}$ [K]', fontsize=16)
#print('Minimum surface temperature = ', np.min(T_surf_array), 'K')
plt.axvspan(t_array[0]/year/1.0e9, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

ax4 = plt.subplot(2, 2, 4)
plt.loglog(t_array/year/1.0e9, EL_array, linewidth=3, color='r', linestyle='-.', label='energy-limited')
plt.loglog(t_array/year/1.0e9, DL_array, linewidth=3, color='g', linestyle='--', label='diffusion-limited')
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#Plot in bold the actual loss prescription being followed, based on TOA flux runaway greenhouse limit.
plt.loglog(t_array/year/1.0e9, loss_array, linewidth=3, color='k', label='loss to space')
plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel('Escape rate [kg/s]', fontsize=16)
plt.legend(fontsize=16, loc='upper right')
plt.axvspan(t_array[0]/year/1.0e9, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

#plt.savefig('PostMO_TOAFlux_LossRates_6TO_MidHZ_50Myr_loglog.pdf', bbox_inches='tight')

In [ ]:
# Plot TOA flux over time, and a horizontal line with Goldblatt/Watson limit of 300 W/m^2.
# First, find where runaway greenhouse ends, to be plotted as a vertical line.
for idx in range(0, len(t_array)):
    
    if TOA_flux_array[idx]/4. < 325./(1.-alb):
        
        exit_time = t_array[idx]
        exit_idx = idx
        break

%matplotlib inline
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(14,10))
plt.tight_layout()
ax = [fig.add_subplot(2,2,i+1) for i in range(4)]
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(2, 2, 1)
plt.loglog(t_MO_array/year/1.0e9, TOA_flux_MO_array/4, linewidth=3, color='b')
plt.loglog(t_array/year/1.0e9, TOA_flux_array/4, linewidth=3, color='b') #divide by 4 for sphere
plt.axhline(325./(1.-alb), color='k', linestyle='--', label='Simpson-Nakajima limit')
plt.text(0.00002, (325./(1.-alb))+30, 'Simpson-Nakajima limit', color='k', fontsize=15)
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.xticks([],[])
plt.ylabel(r'TOA Flux/4 [W/m$^2$]', fontsize=16)
#plt.legend(fontsize=14, loc='upper right')
plt.axvspan(1.0e-5, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

ax2 = plt.subplot(2, 2, 2)
plt.loglog(t_MO_array/year/1.0e9, F_XUV_MO_array, linewidth=3, color='k')
plt.loglog(t_array/year/1.0e9, F_XUV_array, linewidth=3, color='k')
plt.axvline(t_sat/year/1.0e9, linestyle='--', linewidth=2, color='k')
plt.text(0.75, 0.5, r'$t_{{\mathrm{{sat}}}} = 1$ Gyr', color='k', rotation=90, rotation_mode='anchor', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.xticks([],[])
plt.ylabel(r'$F_{{\mathrm{{XUV}}}}$ [W/m$^2$]', fontsize=16, labelpad=2.0)
plt.axvspan(1.0e-5, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

ax3 = plt.subplot(2, 2, 3)
plt.loglog(t_MO_array/year/1.0e9, T_surf_MO_array, linewidth=3, color='k')
plt.loglog(t_array/year/1.0e9, T_surf_array, linewidth=3, color='k')
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.xlabel('Time [Gyr]', fontsize=16)
plt.ylabel(r'$T_{{\mathrm{{surf}}}}$ [K]', fontsize=16)
#print('Minimum surface temperature = ', np.min(T_surf_array), 'K')
plt.axvspan(1.0e-5, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

ax4 = plt.subplot(2, 2, 4)
plt.loglog(t_MO_array/year/1.0e9, EL_MO_array, linewidth=3, color='r', linestyle='-.')
plt.loglog(t_MO_array/year/1.0e9, DL_MO_array, linewidth=3, color='g', linestyle='--')
plt.loglog(t_array/year/1.0e9, EL_array, linewidth=3, color='r', linestyle='-.', label='energy-limited')
plt.loglog(t_array/year/1.0e9, DL_array, linewidth=3, color='g', linestyle='--', label='diffusion-limited')
#plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#Plot in bold the actual loss prescription being followed, based on TOA flux runaway greenhouse limit.
plt.loglog(t_MO_array/year/1.0e9, loss_MO_array, linewidth=3, color='k')
plt.loglog(t_array/year/1.0e9, loss_array, linewidth=3, color='k', label='loss to space')
plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([1.0e-5, 5.0])
plt.ylabel('Escape rate [kg/s]', fontsize=16)
plt.legend(fontsize=14, loc='center left')
plt.axvspan(1.0e-5, t_array[exit_idx]/year/1.0e9, alpha=0.2, color='grey')

#plt.savefig('Temporal_TOAFlux_LossRates_6TO_MidHZ_50Myr_loglog.pdf', bbox_inches='tight')

In [ ]:
# Plot TOA flux over time, and a horizontal line with Goldblatt/Watson limit of 300 W/m^2.
# First, find where runaway greenhouse ends, to be plotted as a vertical line.
for idx in range(0, len(t_array)):
    
    if TOA_flux_array[idx]/4. < 325./(1.-alb):
        
        exit_time = t_array[idx]
        exit_idx = idx
        break

%matplotlib inline
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(8.5,14))
plt.tight_layout()
ax = [fig.add_subplot(3,1,i+1) for i in range(3)]
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(3, 1, 1)
plt.semilogx(t_array/year/1.0e9, TOA_flux_array/4, linewidth=3, color='b') #divide by 4 for sphere
plt.axhline(325./(1.-alb), color='k', linestyle='--', label='Simpson-Nakajima limit')
plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'TOA Flux/4 [W/m$^2$]', fontsize=16)
plt.legend(fontsize=16, loc='center right')

ax2 = plt.subplot(3, 1, 2)
plt.semilogx(t_array/year/1.0e9, EL_array, linewidth=3, color='r', linestyle='--', label='energy-limited')
plt.semilogx(t_array/year/1.0e9, DL_array, linewidth=3, color='g', linestyle='--', label='diffusion-limited')
plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
#Plot in bold the actual loss prescription being followed, based on TOA flux runaway greenhouse limit.
plt.semilogx(t_array/year/1.0e9, loss_array, linewidth=3, color='k', label='loss to space')
plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel('Escape rate [kg/s]', fontsize=16)
plt.legend(fontsize=16, loc='upper right')

ax3 = plt.subplot(3, 1, 3)
plt.semilogx(t_array/year/1.0e9, T_surf_array, linewidth=3)
plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([tau_MO/1.0e9, 5.0])
plt.ylabel(r'$T_{{\mathrm{{surf}}}}$ [K]', fontsize=16)
#print('Minimum surface temperature = ', np.min(T_surf_array), 'K')

#plt.savefig('PostMO_TOAFlux_LossRates_6TO_MidHZ_50Myr_linear.pdf', bbox_inches='tight')

## Plot simulation results, colour-coded based on surface water regimes.

In [ ]:
# Mark where surface becomes desiccated, then where it begins recovering.
tolerance = 1.29e16

dune_first_idx = -1
for ldx in range(0,len(W_s_array)):
    if W_s_array[ldx] <= (0.01*1.4e21):
        dune_first_idx = ldx
        break

desiccated_idx = -1
if dune_first_idx >= 0:
    for idx in range(dune_first_idx+1,len(W_s_array)):
        if W_s_array[idx] <= tolerance:
            desiccated_idx = idx
            break
        
recover_idx = -1
if desiccated_idx >= 0:
    for jdx in range(desiccated_idx+1,len(W_s_array)):
        if W_s_array[jdx] >= tolerance:
            recover_idx = jdx
            break
                     
dune_idx = -1
if recover_idx >= 0:
    for kdx in range(recover_idx+1,len(W_s_array)):
        if W_s_array[kdx] >= (0.01*1.4e21):
            dune_idx = kdx
            break
            
print(desiccated_idx, recover_idx)

In [ ]:
%matplotlib inline
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
fig = plt.figure(figsize=(8.5,15))
plt.tight_layout()
ax = [fig.add_subplot(3,1,i+1) for i in range(3)]
fig.subplots_adjust(hspace=0)

###### WATER INVENTORY VS. TIME ######
ax1 = plt.subplot(3,1,1)
#plt.loglog(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
#plt.loglog(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.loglog(t_array/year/1.0e9, W_m_array/1.4e21, linewidth=3, color='r', linestyle=':', label='Mantle')
plt.loglog(t_array/year/1.0e9, W_s_array/1.4e21, linewidth=3, color='b', linestyle='--', label='Surface')
plt.axhline(y=1.29e16/1.4e21, color='k', linestyle=':',linewidth=3)
plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.', label='End RG')

# FILL IN REGION WHERE IT IS EARTH-LIKE
plt.axvspan(1.0e-4,t_array[dune_first_idx]/year/1.0e9,alpha=0.5,color='green')
# FILL IN REGION WHERE IT FIRST BECOMES DUNE
plt.axvspan(t_array[dune_first_idx]/year/1.0e9, t_array[desiccated_idx]/year/1.0e9, alpha=0.5, color='chocolate')
#FILL-IN REGION WHERE IT IS DESICCATED
plt.axvspan(t_array[desiccated_idx]/year/1.0e9, t_array[recover_idx]/year/1.0e9, alpha=0.5, color='grey')
# FILL IN REGION WHERE IT IS A DUNE PLANET
plt.axvspan(t_array[recover_idx]/year/1.0e9, t_array[dune_idx]/year/1.0e9, alpha=0.5, color='chocolate')

plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([0.05, 5.0])
plt.ylabel(r'Water Mass [TO]', fontsize=16)
plt.legend(loc='center left', fontsize=16)

####### CYCLING & LOSS RATES VS. TIME #######
ax2 = plt.subplot(3,1,2)
#plt.loglog(t_array/year/1.0e9, degas_array/1.4e21*year*1.0e9, linewidth=3, color='b', linestyle='--', label='Degassing')
#plt.loglog(t_array/year/1.0e9, regas_array/1.4e21*year*1.0e9, linewidth=3, color='r', linestyle=':', label='Regassing')
#plt.loglog(t_array/year/1.0e9, loss_array/1.4e21*year*1.0e9, linewidth=3, color='k', linestyle='-.', label='Loss to Space')
plt.semilogx(t_array/year/1.0e9, degas_array/1.4e21*year*1.0e9, linewidth=3, color='b', linestyle='--', label='Degassing')
plt.semilogx(t_array/year/1.0e9, regas_array/1.4e21*year*1.0e9, linewidth=3, color='r', linestyle=':', label='Regassing')
plt.semilogx(t_array/year/1.0e9, loss_array/1.4e21*year*1.0e9, linewidth=3, color='k', linestyle='-.', label='Loss to Space')
plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')

# FILL IN REGION WHERE IT IS EARTH-LIKE
plt.axvspan(1.0e-4,t_array[dune_first_idx]/year/1.0e9,alpha=0.5,color='green')
# FILL IN REGION WHERE IT FIRST BECOMES DUNE
plt.axvspan(t_array[dune_first_idx]/year/1.0e9, t_array[desiccated_idx]/year/1.0e9, alpha=0.5, color='chocolate')
#FILL-IN REGION WHERE IT IS DESICCATED
plt.axvspan(t_array[desiccated_idx]/year/1.0e9, t_array[recover_idx]/year/1.0e9, alpha=0.5, color='grey')
# FILL IN REGION WHERE IT IS A DUNE PLANET
plt.axvspan(t_array[recover_idx]/year/1.0e9, t_array[dune_idx]/year/1.0e9, alpha=0.5, color='chocolate')

plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([0.05, 5.0])
plt.ylabel(r'Cycling Rate [TO/Gyr]', fontsize=16)
plt.legend(loc='center left', fontsize=16)
#plt.ylim([-0.05,650.])
#plt.legend(bbox_to_anchor=(0.0, 1.0), ncol=2, fontsize=16)

###### SURFACE TEMPERATURE VS. TIME #######
ax3 = plt.subplot(3,1,3)
plt.semilogx(t_array/year/1.0e9, T_surf_array, color='k', linewidth=3)

# FILL IN REGION WHERE IT IS EARTH-LIKE
plt.axvspan(1.0e-4,t_array[dune_first_idx]/year/1.0e9,alpha=0.5,color='green')
# FILL IN REGION WHERE IT FIRST BECOMES DUNE
plt.axvspan(t_array[dune_first_idx]/year/1.0e9, t_array[desiccated_idx]/year/1.0e9, alpha=0.5, color='chocolate')
#FILL-IN REGION WHERE IT IS DESICCATED
plt.axvspan(t_array[desiccated_idx]/year/1.0e9, t_array[recover_idx]/year/1.0e9, alpha=0.5, color='grey')
# FILL IN REGION WHERE IT IS A DUNE PLANET
plt.axvspan(t_array[recover_idx]/year/1.0e9, t_array[dune_idx]/year/1.0e9, alpha=0.5, color='chocolate')

plt.axvline(exit_time/year/1.0e9, color='k', linestyle='-.')
plt.xlabel('Time [Gyr]', fontsize=16)
plt.xlim([0.05, 5.0])
plt.ylabel(r'$T_{{\mathrm{{surf}}}}$ [K]', fontsize=16)

#plt.savefig('Cycling_Loss_1DAtmos_2TOtotal_coloured_linearyaxis.pdf', bbox_inches='tight')
#plt.savefig('Cycling_Loss_1DAtmos_1TO_loglog_Psurf_0d01bar.pdf', bbox_inches='tight')
#plt.savefig('Cycling_Loss_1DAtmos_1TO_loglog_Psurf_270bar.pdf', bbox_inches='tight')

In [ ]:
print('Planet leaves runaway greenhouse at ', (exit_time+50.0e6*year)/year/1.0e9, ' Gyr')

## Create a grid figure to display results of ALL runs:
### Each panel = total initial water (2, 4, 6, 8 TO)
### y-axis: tau_MO (10, 50, 100 Myr)
### x-axis: HZ location (Inner, Mid, Outer)
### Points: big hollow dashed circle = initial TOTAL water in magma ocean; small solid circle = final TOTAL water, coloured based on final SURFACE water

In [ ]:
# If desiccated at end, black X.
# Size of circles indicate the amount of water. (bigger = more water)

%matplotlib inline

plt.rc('font', family='serif')
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)
fig = plt.figure(figsize=(8,10))
#ax1 = fig.add_subplot(111)
#ax2 = ax1.twiny()
#ax3 = ax1.twinx()
#plt.tight_layout()
#fig.subplots_adjust(hspace=0, wspace=0)

tolerance = 1.29e16 #anything below this, no water (<~0.00001 TO)

# For figure to be nice and clean, need to have temporary indices, and add labels after plotting.
water_y = np.array([1,2,3,4]) #need to label [2, 4, 6, 8] Earth Oceans
HZ_x = np.array([1,2,3]) #need to label [Inner, Mid, Outer] HZ

# Combine water inventories into TOTAL initial and TOTAL final:
W_initial_total = M_MO_initial_all + M_SM_initial_all + M_atmo_initial_all
W_final_total = W_m_final_all + W_s_final_all
    
for jdx in range(0,len(HZ_x)):
        
    for kdx in range(0,len(water_y)):
            
        #ax1 = plt.subplot(2,2,idx+1)

        # ALL WATER BEGINS DISSOLVED IN MAGMA OCEAN, FOR EACH SIMULATION!!!
        init_color = 'orange'
            
        # Choose marker size based on initial water. XXXX FIGURE OUT A GOOD WAY TO DO THIS XXXX
        init_size = 10000.

        # Choose corresponding colour for final values, BASED ON SURFACE WATER INVENTORY.
        if W_s_final_all[kdx,jdx] < tolerance: #no surface water
            fin_color = 'black'
            fin_mark = 'x'
        elif W_s_final_all[kdx,jdx] > tolerance and W_s_final_all[kdx,jdx] < 0.01*1.4e21:
            fin_color = 'chocolate' #Dune planet
            fin_mark = 'o'
        elif W_s_final_all[kdx,jdx] >= 0.01*1.4e21 and W_s_final_all[kdx,jdx] < 10.0*1.4e21:
            fin_color = 'green' #habitable
            fin_mark = 'o'
        elif W_s_final_all[kdx,jdx] >= 10.0*1.4e21:
            fin_color = 'blue' #waterworld
            fin_mark = 'o'
    
        # Choose marker size based on final TOTAL water.
        if fin_color == 'black':
            fin_size = 4000.
            lws = 5.
        else:
            fin_size = init_size*((W_final_total[kdx,jdx]/W_initial_total[kdx,jdx])**0.5)
            lws = 3.
    
        # Plot initial and final water amounts.
        #plt.plot(HZ_x[jdx], tau_y[kdx], fillstyle='none', color=init_color, marker='o', linestyle='', markersize=init_size)
        #plt.plot(HZ_x[jdx], tau_y[kdx], fillstyle='full', color=fin_color, marker=fin_mark, linestyle='', markersize=fin_size)
        plt.scatter(HZ_x[jdx], water_y[kdx], s=fin_size, facecolors=fin_color, edgecolors='k', marker=fin_mark, linestyle='-',alpha=1, linewidths=3, zorder=3)
        plt.scatter(HZ_x[jdx], water_y[kdx], s=init_size, facecolors='none', edgecolors='orange',  linestyle='--', marker='o',alpha=1, linewidths=3, zorder=3)

labels_x = ['Inner', 'Mid', 'Outer']
plt.xticks(HZ_x, labels_x, rotation='vertical', fontsize=24)
labels_y = ['2', '4', '6', '8']
plt.yticks(water_y, labels_y, fontsize=24)
plt.xlim(0.55, 3.45)
plt.ylim(0.55, 4.45)
plt.xlabel('Location within HZ', fontsize=24)
plt.ylabel('Initial Water Inventory [Earth Oceans]', fontsize=24)
        
# Handles for legend only
#ax1.plot(100., 100., color='k', fillstyle='none', marker='*', linestyle='', markersize=12, label='initial inventory')
#ax1.plot(100., 100., color='k', marker='s', linestyle='', markersize=8, label='final inventory')
#ax1.plot(100., 100., color='k', marker='x', linestyle='', markersize=8, label='surface desiccated')
#ax1.plot(100., 100., color='chocolate', marker='s', linestyle='', markersize=8, label='Dune planet')
#ax1.plot(100., 100., color='green', marker='s', linestyle='', markersize=8, label='Earth-like')
#ax1.plot(100., 100., color='b', marker='s', linestyle='', markersize=8, label='waterworld')

#plt.grid(which='both')
#ax1.legend(bbox_to_anchor=(1.15, 0.8), fontsize=16)
#plt.legend(loc='lower right', fontsize=16) 
#plt.savefig('Model2_Grid_Parameter_Space_M8star_MO_Cycling_C0d001.pdf', bbox_inches='tight')

In [ ]:
#print(tau_y, HZ_x)
#print(np.shape(W_final_total))
print('>tolerance: ', W_s_final_all>tolerance)
print('<Earth-like: ', W_s_final_all<0.01*1.4e21)

In [ ]:
# Double-check if any final surface water inventories fall within the Dune planet regime.
dunecheck1 = W_s_final_all>tolerance
dunecheck2 = W_s_final_all<0.01*1.4e21
print(np.multiply(dunecheck1,dunecheck2))